<a href="https://colab.research.google.com/github/DDDS18-GTFS/ddds.18.capstone/blob/dev.Andrew/GTFS_DB_Static_Filter_Sort_v9_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ABC note 250726-1715: I have some concerns about the anomaly placement, but I also haven't worked with a multi-day dataset previously, so I need to do some more testing.


#Load the Libraries

In [1]:
#Required Libraries
import pandas as pd
import numpy as np

# # Install if needed
# !pip install -q ipywidgets

import ipywidgets as widgets
from IPython.display import display, clear_output

from datetime import datetime
import zipfile
import urllib.request

from shapely.geometry import Point, LineString
from geopy.distance import geodesic

from folium import Map, FeatureGroup, CircleMarker, PolyLine, Marker, Icon, LayerControl
from matplotlib import colors as mcolors
import matplotlib.pyplot as plt
import folium

!pip install psycopg2-binary pandas sqlalchemy

import psycopg2
from sqlalchemy import create_engine
import os
from google.colab import userdata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 41.1 MB/s eta 0:00:00


In [2]:
# Database connection parameters
DB_NAME = "abq-transit-db"

# You'll need to store your database password/paramenters in Colab Secrets
DB_HOST = userdata.get('DB_HOST_2')
DB_PORT = userdata.get('DB_PORT')
DB_PASSWORD = userdata.get('DB_PASSWORD')
DB_USER = userdata.get("DB_USER")


# Create connection string
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create SQLAlchemy engine
engine = create_engine(connection_string)

In [3]:
#Collect Static zip
def download_file_urllib(url, local_filename):
    """
    Downloads a file from a URL using urllib.request and saves it locally.
    """
    try:
        urllib.request.urlretrieve(url, local_filename)
        print(f"File downloaded successfully to: {local_filename}")
    except Exception as e:
        print(f"Error downloading file: {e}")

# Example usage:
file_url = "https://data.cabq.gov/transit/gtfs/google_transit.zip"  # Replace with your URL
output_filename = "google_transit.zip"
download_file_urllib(file_url, output_filename)

File downloaded successfully to: google_transit.zip


In [4]:
# Select Parameters for Anomaly Detection
# This section sets threshholds for classifying anomalous transit events.

# --- Use meters for user input ---
# The jump_thresh_m_widget creates a sliding bar for selecting how much distance there must be between
# two consecutive positions for a single vehicle to have gone 'unusually far' or 'jumped'.
jump_thresh_m_widget = widgets.IntSlider(
    value=500,
    min=50,
    max=2000,
    step=50,
    description='Jump Distance',
    layout=widgets.Layout(width='60%')
)

# Other anomaly widgets
# The disappear_thresh_widget creates a sliding bar for selecting how many consecutive seconds  must pass
# without a vehicle transmitting data before we consider that vehicle to have been gone 'unusually long'.
disappear_thresh_widget = widgets.IntSlider(value=300, min=60, max=1800, step=30, description='Time Gap')

# The min_jump_widget creates a sliding bar for selecting how many 'jumps' a vehicle must make before
# we consider the jumping to be 'unusual'.
# SUGGESTION: can we make this based off of jump points as a percentage of total recorded points a required number of consecutive jumps?
# As we use more data, any set value of total jumps will become more and more easily met.
min_jump_widget = widgets.IntSlider(value=2, min=1, max=10, step=1, description='Min Jumps')

# The speed_thresh_widget creates a sliding bar for selecting how fast a vihicke must be going for us
# to consider that vehicle as 'unusually fast'.
speed_thresh_widget = widgets.IntSlider(value=70, min=10, max=100, step=5, description='Speed')

# motion to depricate this widget
reversal_thresh_widget = widgets.IntSlider(value=120, min=60, max=180, step=5, description='Heading Δ°')

# --- Stuck vehicle detection ---
# The stuck_speed_widget creates a sliding bar for selecting how slow a vehicle must be going
# for us to consider it as effectively unmoving.
stuck_speed_widget = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=5.0,
    step=0.1,
    description='Stuck Speed',
    layout=widgets.Layout(width='60%')
)

# The stuck_window_widget creates a sliding bar for selecting how many consecutive Stuck Speed
# events a vehicle must have before it is considered to have been still for an unusually long period.
stuck_window_widget = widgets.IntSlider(
    value=4,
    min=1,
    max=20,
    step=1,
    description='Window Size',
    layout=widgets.Layout(width='60%')
)

# --- Repeated point tolerance ---
# motion to depricate this widget
repeat_tolerance_widget = widgets.FloatLogSlider(
    value=1e-5,
    base=10,
    min=-7,  # 1e-7
    max=-3,  # 1e-3
    step=0.1,
    description='Repeat Tolerance',
    layout=widgets.Layout(width='60%')
)

# --- Early appearance margin ---
# motion to depricate this widget
early_margin_widget = widgets.IntSlider(
    value=30,
    min=0,
    max=600,
    step=10,
    description='Early Margin',
    layout=widgets.Layout(width='60%')
)

# --- Off-route buffer ---
# The offroute_buffer_widget creates a sliding bar for selecting how many meters
# a vehicle must be off of its expected route before we actually label it as off route
offroute_buffer_widget = widgets.IntSlider(
    value=50,
    min=10,
    max=500,
    step=10,
    description='Off-Route Buffer',
    layout=widgets.Layout(width='60%')
)

# Labels
jump_label = widgets.Label(value="(meters)")
disappear_label = widgets.Label(value="(seconds)")
min_jump_label = widgets.Label(value="(count)")
speed_label = widgets.Label(value="(mph)")
reversal_label = widgets.Label(value="(degrees of reversal)")
stuck_speed_label = widgets.Label(value="(mph)")
stuck_window_label = widgets.Label(value="(frames in rolling window)")
repeat_tolerance_label = widgets.Label(value="(decimal degrees)")
early_margin_label = widgets.Label(value="(seconds)")
offroute_buffer_label = widgets.Label(value="(meters)")

# Assemble UI layout
slider_widgets = widgets.VBox([
    widgets.HBox([jump_thresh_m_widget, jump_label]),
    widgets.HBox([disappear_thresh_widget, disappear_label]),
    widgets.HBox([min_jump_widget, min_jump_label]),
    widgets.HBox([speed_thresh_widget, speed_label]),
    widgets.HBox([reversal_thresh_widget, reversal_label]),
    widgets.HBox([stuck_speed_widget, stuck_speed_label]),
    widgets.HBox([stuck_window_widget, stuck_window_label]),
    widgets.HBox([repeat_tolerance_widget, repeat_tolerance_label]),
    widgets.HBox([early_margin_widget, early_margin_label]),
    widgets.HBox([offroute_buffer_widget, offroute_buffer_label]),
])

# Button and save logic
submit_button = widgets.Button(description="Save Parameters", button_style='primary')
anomaly_params = {}

def save_params(b):
    '''
    save_params displays widgets that allow you to set up and save anomaly detection threshholds.
    '''
    clear_output(wait=True)
    display(slider_widgets, submit_button)

    global anomaly_params
    jump_m = jump_thresh_m_widget.value
    jump_deg = jump_m / 111000  # Convert meters → degrees

    anomaly_params = {
        "JUMP_DISTANCE_THRESHOLD": jump_deg,
        "JUMP_DISTANCE_METERS": jump_m,
        "DISAPPEARANCE_TIME_THRESHOLD": disappear_thresh_widget.value,
        "MIN_JUMP_COUNT_PER_VEHICLE": min_jump_widget.value,
        "SPEED_LIMIT_MPH": speed_thresh_widget.value,
        "REVERSAL_HEADING_THRESHOLD": reversal_thresh_widget.value,
        "STUCK_SPEED_MPH": stuck_speed_widget.value,
        "STUCK_WINDOW_SIZE": stuck_window_widget.value,
        "REPEATED_COORD_TOLERANCE": repeat_tolerance_widget.value,
        "EARLY_APPEARANCE_MARGIN_SEC": early_margin_widget.value,
        "OFF_ROUTE_BUFFER_M": offroute_buffer_widget.value,
    }

    print("✅ Anomaly detection parameters set:")
    for k, v in anomaly_params.items():
        print(f"  {k}: {v}")

submit_button.on_click(save_params)

# Display interface
display(slider_widgets, submit_button)


Button(button_style='primary', description='Save Parameters', style=ButtonStyle())

✅ Anomaly detection parameters set:
  JUMP_DISTANCE_THRESHOLD: 0.0045045045045045045
  JUMP_DISTANCE_METERS: 500
  DISAPPEARANCE_TIME_THRESHOLD: 300
  MIN_JUMP_COUNT_PER_VEHICLE: 2
  SPEED_LIMIT_MPH: 70
  REVERSAL_HEADING_THRESHOLD: 120
  STUCK_SPEED_MPH: 1.0
  STUCK_WINDOW_SIZE: 4
  REPEATED_COORD_TOLERANCE: 1e-05
  EARLY_APPEARANCE_MARGIN_SEC: 30
  OFF_ROUTE_BUFFER_M: 50


#LOAD PICKLE

In [ ]:
# This cell should be used when loading pickle files as opposed to using the database
#import pickle

# Load a pickle file of a dataframe which contains our historic transit data.
#with open("captures.p", "rb") as f:
#    captured_data = pickle.load(f)


In [5]:
with engine.connect() as connection:
    captured_data = pd.read_sql_query(
        '''
        SELECT
          id, snapshot_id, msg_time, timestamp_collected, vehicle_id, ST_X("vehicle_snapshots"."location") AS longitude, ST_Y("vehicle_snapshots"."location") AS latitude, heading, speed_mph,
          route_short_name, trip_id, next_stop_id, next_stop_id, next_stop_name,
          next_stop_sched_time
        FROM
          vehicle_snapshots
        WHERE
          DATE("timestamp_collected") = (CURRENT_DATE - INTERVAL '1 DAY')
        ORDER BY
          timestamp_collected DESC;
        ''', connection)

captured_data.head()

,id,snapshot_id,msg_time,timestamp_collected,vehicle_id,longitude,latitude,heading,speed_mph,route_short_name,trip_id,next_stop_id,next_stop_id,next_stop_name,next_stop_sched_time
0,3896308,8596,16:32:39,2025-08-04 23:59:35.721861+00:00,757,-106.622787,35.071476,98.0,0.00,Off Duty,0,0,0,Off Duty,16:32:39
1,3896307,8596,10:00:24,2025-08-04 23:59:35.721861+00:00,756,-106.734251,35.089362,359.0,11.27,Off Duty,0,0,0,No Data,10:00:24
2,3896306,8596,08:13:18,2025-08-04 23:59:35.721861+00:00,755,-106.733507,35.089424,118.0,0.00,Off Duty,0,0,0,No Data,08:13:18
3,3896305,8596,15:28:16,2025-08-04 23:59:35.721861+00:00,754,-106.734137,35.089393,181.0,0.00,Off Duty,0,0,0,No Data,15:28:16
4,3896304,8596,12:55:16,2025-08-04 23:59:35.721861+00:00,753,-106.732687,35.089868,0.0,0.00,Off Duty,0,0,0,No Data,12:55:16


##Data Validation

In [6]:
cd_df = captured_data
cd_df.size

14824680

In [ ]:
# cd_df.head(20)

In [ ]:
# # Display full list of column names
# print("Number of columns:", len(cd_df.columns))
# print("Column names:\n", cd_df.columns.tolist())

In [ ]:
# cd_df.head()

In [7]:
# Convert timestamp_collected & msg_time to datetime
cd_df['timestamp_collected'] = pd.to_datetime(cd_df['timestamp_collected'], errors='coerce')
cd_df['msg_time'] = pd.to_datetime(cd_df['msg_time'], errors='coerce')

# 1. Shape and Column Overview
print("Shape:", cd_df.shape)
print("Columns:", cd_df.columns.tolist())

# 2. Null/Missing Value Counts
print("\nMissing Values:\n", cd_df.isnull().sum())

# 3. Duplicate Detection
duplicates = cd_df.duplicated().sum()
print("\nDuplicate rows:", duplicates)

# 4. Timestamp Range & msg_time range
print("\nTimestamp Range:")
print("Min:", cd_df['timestamp_collected'].min())
print("Max:", cd_df['timestamp_collected'].max())
print("Duration:", cd_df['timestamp_collected'].max() - cd_df['timestamp_collected'].min())

print("\nmsg_time:")
print("Min:", cd_df['msg_time'].min())
print("Max:", cd_df['msg_time'].max())
print("Range:", cd_df['msg_time'].max() - cd_df['msg_time'].min())

# 5. Latitude/Longitude Range
print("\nLatitude Range:", cd_df['latitude'].min(), "to", cd_df['latitude'].max())
print("Longitude Range:", cd_df['longitude'].min(), "to", cd_df['longitude'].max())

# 6. Speed Summary
print("\nSpeed Summary:\n", cd_df['speed_mph'].describe())


/tmp/ipython-input-3544357411.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cd_df['msg_time'] = pd.to_datetime(cd_df['msg_time'], errors='coerce')


Shape: (988312, 15)
Columns: ['id', 'snapshot_id', 'msg_time', 'timestamp_collected', 'vehicle_id', 'longitude', 'latitude', 'heading', 'speed_mph', 'route_short_name', 'trip_id', 'next_stop_id', 'next_stop_id', 'next_stop_name', 'next_stop_sched_time']

Missing Values:
 id                      0
snapshot_id             0
msg_time                0
timestamp_collected     0
vehicle_id              0
longitude               0
latitude                0
heading                 0
speed_mph               0
route_short_name        0
trip_id                 0
next_stop_id            0
next_stop_id            0
next_stop_name          0
next_stop_sched_time    0
dtype: int64

Duplicate rows: 0

Timestamp Range:
Min: 2025-08-04 00:00:09.122007+00:00
Max: 2025-08-04 23:59:35.721861+00:00
Duration: 0 days 23:59:26.599854

msg_time:
Min: 2025-08-05 00:01:18
Max: 2025-08-05 23:59:49
Range: 0 days 23:58:31

Latitude Range: 0.0 to 39.1386265
Longitude Range: -106.7925358 to 0.0

Speed Summary:
 count 

#####msg_time issue
msg_time is defaulting to today's date when attempting to perform arithmetic on the values
I think we need to look at historical data to determine the datetime for the msg_time

In [ ]:
# # Plot the histogram and get the counts and bin edges
# counts, bins, patches = plt.hist(cd_df['longitude'], bins=2, rwidth=0.8)

# # Add labels and title
# plt.xlabel("Value")
# plt.ylabel("Count")
# plt.title("Histogram of Values with Counts per Bin")

# # Display the plot
# plt.show()

# # You can also access the counts and bin edges directly:
# print("Bin counts:", counts)
# print("Bin edges:", bins)

In [ ]:
# # Plot the histogram and get the counts and bin edges
# counts, bins, patches = plt.hist(cd_df['latitude'], bins=2, rwidth=0.8)

# # Add labels and title
# plt.xlabel("Value")
# plt.ylabel("Count")
# plt.title("Histogram of Values with Counts per Bin")

# # Display the plot
# plt.show()

# # You can also access the counts and bin edges directly:
# print("Bin counts:", counts)
# print("Bin edges:", bins)

In [8]:
# Select rows where 'longitude' and 'latitude' have the same value
rows_with_same_values = cd_df[cd_df['longitude'] == cd_df['latitude']]
rows_without_same_values = cd_df[cd_df['longitude'] != cd_df['latitude']]

# Print the resulting DataFrame
print(rows_with_same_values)

             id  snapshot_id            msg_time  \
258     3896461         8596 2025-08-05 20:00:59   
473     3896117         8595 2025-08-05 20:00:59   
723     3895773         8594 2025-08-05 20:00:59   
1067    3895429         8593 2025-08-05 20:00:59   
1412    3895084         8592 2025-08-05 20:00:59   
...         ...          ...                 ...   
986633  2909863         5726 2025-08-05 20:00:59   
986977  2909519         5725 2025-08-05 20:00:59   
987321  2909175         5724 2025-08-05 20:00:59   
987665  2908831         5723 2025-08-05 20:00:59   
988004  2908487         5722 2025-08-05 20:00:59   

                    timestamp_collected vehicle_id  longitude  latitude  \
258    2025-08-04 23:59:35.721861+00:00       4001        0.0       0.0   
473    2025-08-04 23:59:05.421088+00:00       4001        0.0       0.0   
723    2025-08-04 23:58:37.522141+00:00       4001        0.0       0.0   
1067   2025-08-04 23:58:05.621510+00:00       4001        0.0       0.0   


In [ ]:
# rows_without_same_values.shape

In [ ]:
# cd_df.shape

####*.v8.2 update
We're removing the off-duty rows where Lat = Long = 0

In [9]:
cd_df_copy = cd_df.copy()
cd_df_copy.shape

(988312, 15)

In [10]:
cd_df = cd_df[cd_df['longitude'] != cd_df['latitude']]
cd_df.shape

(985417, 15)

#####Note: Want to consider msg_time that looks active
Could define this as something like msg_time is within a few minutes of timestamp_collected (but this will create false hits for at least a few snapshots each day)

In [ ]:
#Consider adding code to only pick up msg_time that appears active; may need to look at historical data to accomplish this

#Convert code for snapshot to use Pickle

In [11]:
def filter_cd_df(df, start_time=None, end_time=None, routes=None, vehicles=None, bounds=None):
    '''
    Filters a dataframe(df) such that:
    - the "timestamp_collected" column's entries are greater than or equal to the start_time
    - the "timestamp_collected" column's entries are less than or equal to the end_time
    - the "route_short_name" column's entries are in routes
    - the "vehicle_id" column's entries are in vehicles

    Returns the filtered dataframe.
    '''
    df_filtered = df.copy()

    if start_time:
        df_filtered = df_filtered[df_filtered['timestamp_collected'] >= pd.to_datetime(start_time)]
    if end_time:
        df_filtered = df_filtered[df_filtered['timestamp_collected'] <= pd.to_datetime(end_time)]
    if routes:
        df_filtered = df_filtered[df_filtered['route_short_name'].isin(routes)]
    if vehicles:
        df_filtered = df_filtered[df_filtered['vehicle_id'].isin(vehicles)]
    if bounds:
        lat_min, lat_max, lon_min, lon_max = bounds
        df_filtered = df_filtered[
            (df_filtered['latitude'] >= lat_min) & (df_filtered['latitude'] <= lat_max) &
            (df_filtered['longitude'] >= lon_min) & (df_filtered['longitude'] <= lon_max)
        ]

    return df_filtered


In [12]:
cd_df['msg_time'].info()

<class 'pandas.core.series.Series'>
Index: 985417 entries, 0 to 988311
Series name: msg_time
Non-Null Count   Dtype         
--------------   -----         
985417 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 15.0 MB


In [13]:


# --- Widgets ---
# route_widget lets you select which routes you'd like to like to plot in folium from a menu.
route_widget = widgets.SelectMultiple(
    options=sorted(cd_df['route_short_name'].unique()),
    description='Routes',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)
# vehicle_widget lets you select which vehicles' data you'd like to plot in folium from a menu.
vehicle_widget = widgets.SelectMultiple(
    options=sorted(cd_df['vehicle_id'].unique()),
    description='Vehicles',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)
# start_widget lets you select the earliest date you'd like to use data from.
start_widget = widgets.DatePicker(
    description='Start Date',
    value=cd_df['timestamp_collected'].min().date()
)
# end_widget lets you select the latest date you'd like to use data from.
end_widget = widgets.DatePicker(
    description='End Date',
    value=cd_df['timestamp_collected'].max().date()
)

# hour_slider creates a set of sliding bars for selecting the earliest and latest
# point of the day that you'd like to use data from.
hour_slider = widgets.IntRangeSlider(
    value=[0, 23],
    min=0,
    max=23,
    step=1,
    description='Hour Range',
    layout=widgets.Layout(width='60%')
)

# Makes a button, that will later be used to apply filters.
filter_button = widgets.Button(description='Apply Filters', button_style='primary')

# --- Global to store result ---
cd_df_filtered = pd.DataFrame()

# --- Filtering callback ---
def apply_filters(b):
    '''
    Uses previously defined widgets to filter a dataframe of historical transit data
    by rout_short_name, vehicle_id, and timestamp collected.
    Prints the number of filtered rows and displays the first 10 rows of the filtered dataframe.
    '''
    global cd_df_filtered
    clear_output(wait=True)
    display(route_widget, vehicle_widget, start_widget, end_widget, hour_slider, filter_button)

    df = cd_df.copy()

    # Apply filters
    if route_widget.value:
        df = df[df['route_short_name'].isin(route_widget.value)]
    if vehicle_widget.value:
        df = df[df['vehicle_id'].isin(vehicle_widget.value)]

    start_dt = pd.to_datetime(start_widget.value).tz_localize('UTC')
    end_dt = (pd.to_datetime(end_widget.value) + pd.Timedelta(days=1)).tz_localize('UTC')
    df = df[(df['timestamp_collected'] >= start_dt) &
            (df['timestamp_collected'] < end_dt)]

    hr_start, hr_end = hour_slider.value
    df = df[(df['timestamp_collected'].dt.hour >= hr_start) &
            (df['timestamp_collected'].dt.hour <= hr_end)]

    # Store result globally
    cd_df_filtered = df

    print(f"✅ Filtered {len(df)} rows.")
    display(df.head(10))

# Displays a filter button that calls the 'apply_filters' function
filter_button.on_click(apply_filters)

# --- Display UI ---
display(route_widget, vehicle_widget, start_widget, end_widget, hour_slider, filter_button)


SelectMultiple(description='Routes', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, …

SelectMultiple(description='Vehicles', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18…

DatePicker(value=datetime.date(2025, 8, 4), description='Start Date')

DatePicker(value=datetime.date(2025, 8, 4), description='End Date')

IntRangeSlider(value=(0, 23), description='Hour Range', layout=Layout(width='60%'), max=23)

Button(button_style='primary', description='Apply Filters', style=ButtonStyle())

✅ Filtered 985417 rows.


,id,snapshot_id,msg_time,timestamp_collected,vehicle_id,longitude,latitude,heading,speed_mph,route_short_name,trip_id,next_stop_id,next_stop_id,next_stop_name,next_stop_sched_time
0,3896308,8596,2025-08-05 16:32:39,2025-08-04 23:59:35.721861+00:00,757,-106.622787,35.071476,98.0,0.00,Off Duty,0,0,0,Off Duty,16:32:39
1,3896307,8596,2025-08-05 10:00:24,2025-08-04 23:59:35.721861+00:00,756,-106.734251,35.089362,359.0,11.27,Off Duty,0,0,0,No Data,10:00:24
2,3896306,8596,2025-08-05 08:13:18,2025-08-04 23:59:35.721861+00:00,755,-106.733507,35.089424,118.0,0.00,Off Duty,0,0,0,No Data,08:13:18
3,3896305,8596,2025-08-05 15:28:16,2025-08-04 23:59:35.721861+00:00,754,-106.734137,35.089393,181.0,0.00,Off Duty,0,0,0,No Data,15:28:16
4,3896304,8596,2025-08-05 12:55:16,2025-08-04 23:59:35.721861+00:00,753,-106.732687,35.089868,0.0,0.00,Off Duty,0,0,0,No Data,12:55:16
5,3896303,8596,2025-08-05 10:22:25,2025-08-04 23:59:35.721861+00:00,752,-106.623230,35.072577,323.0,0.00,Off Duty,0,0,0,No Data,10:22:25
6,3896302,8596,2025-08-05 16:39:09,2025-08-04 23:59:35.721861+00:00,751,-106.727350,35.086742,239.0,0.00,Off Duty,0,0,0,No Data,16:39:09
7,3896301,8596,2025-08-05 07:30:00,2025-08-04 23:59:35.721861+00:00,750,-106.622454,35.071828,90.0,0.00,Off Duty,0,0,0,No Data,07:30:00
8,3896300,8596,2025-08-05 19:04:34,2025-08-04 23:59:35.721861+00:00,749,-106.733571,35.088586,83.0,0.00,Off Duty,0,0,0,No Data,19:04:34
9,3896299,8596,2025-08-05 13:00:00,2025-08-04 23:59:35.721861+00:00,748,-106.622336,35.071787,90.0,0.00,Off Duty,0,0,0,No Data,13:00:00


In [ ]:
# #Sanity check following filtering
# cd_df_filtered.describe()
# # or
# cd_df_filtered['vehicle_id'].value_counts()


In [14]:
#Pickle replacement
df_new = cd_df_filtered

#Old code:
#✅ Step 1a: Load and Inspect the New Snapshot
# new_snapshot_path = "/content/cabq_gtfs_snapshots_20250722_1415.csv"
# df_new = pd.read_csv(new_snapshot_path)
# df_new.info()
# df_new.head(3)

# #Also print the columns:
# print(df_new.columns.tolist())


In [15]:
#✅ Step 2a: Trip ID Validity
# Counts and displays the total rows and number of rows with invalid trip ids.
# Calculates and displays the percentage of rows with valid trip ids.
df_new["trip_id"] = df_new["trip_id"].astype(str)
invalid_trip_ids = df_new["trip_id"].isin(["0", "Undetermined", "nan", "", "None"]).sum()
total_rows = len(df_new)

print(f"Total rows: {total_rows}")
print(f"Invalid trip_ids: {invalid_trip_ids}")
print(f"Percent valid trip_ids: {100 * (total_rows - invalid_trip_ids) / total_rows:.2f}%")


Total rows: 985417
Invalid trip_ids: 901099
Percent valid trip_ids: 8.56%


In [16]:
#*.v8.2 new calc for finding mostly Off Duty
invalid_route_ids = df_new["route_short_name"].isin(["0", "Undetermined", "nan", "", "None", "Off Duty"]).sum()
print(f"Invalid route_ids: {invalid_route_ids}")
print(f"Percent valid route_ids: {100 * (total_rows - invalid_route_ids) / total_rows:.2f}%")

Invalid route_ids: 890226
Percent valid route_ids: 9.66%


In [17]:
invalid_ids = ["0", "Undetermined", "nan", "", "None", "Off Duty"]
df_clean_test = df_new[~df_new["trip_id"].isin(invalid_ids) & ~df_new["route_short_name"].isin(invalid_ids)].copy()
df_clean_test.shape

(84318, 15)

In [18]:
#✅ Step 2b: Filter invalid trip_ids
invalid_trip_ids = ["0", "Undetermined", "nan", "", "None"]
df_clean_trips = df_new[~df_new["trip_id"].isin(invalid_trip_ids)].copy()
df_clean_trips.shape

(84318, 15)

In [19]:
#So after all that we can see that all of the rows dropped by df_clean_trips cover those dropped by df_clean_test (both invalid trips and routes)
invalid_route_ids = df_clean_trips["route_short_name"].isin(["0", "Undetermined", "nan", "", "None", "Off Duty"]).sum()
print(f"Invalid route_ids: {invalid_route_ids}")
print(f"Percent valid route_ids: {100 * (total_rows - invalid_route_ids) / total_rows:.2f}%")

Invalid route_ids: 0
Percent valid route_ids: 100.00%


In [20]:
df_clean = df_clean_test

In [ ]:
# df_clean.head()

####Cleanup note
We can skip several of the above cells, as dropping the invalid trip_ids also drops the invalid route_short_names

#Load the Static data

In [21]:
#✅ Step 1b: Reload Static GTFS and Normalize It (ensure data types are consistent)

# Adjust if needed — make sure this is the static feed aligned with 2025-07-22
gtfs_zip_path = "/content/google_transit.zip"

with zipfile.ZipFile(gtfs_zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/gtfs_static")

trips = pd.read_csv("/content/gtfs_static/trips.txt", dtype=str)
routes = pd.read_csv("/content/gtfs_static/routes.txt", dtype=str)


In [ ]:
# trips.shape

In [ ]:
# trips.head()

In [ ]:
# trips.describe()

In [ ]:
# routes.shape

In [ ]:
# routes.head()

In [ ]:
# routes.describe(include='all')

In [22]:
#✅ Step 3b: Merge with trips.txt to Get route_id
# Take all of df_clean and add on data from trips if the trip_id matches across both dataframes.
df_with_trips = df_clean.merge(trips, on="trip_id", how="left")


In [ ]:
# df_with_trips.head()

In [ ]:
# df_with_trips.columns

####Merge check
1. Trip-Based Merges (Correct in Isolation)
- df_with_trips = df_clean.merge(trips, on="trip_id", how="left")
- df_full = df_with_trips.merge(routes, on="route_id", how="left")

✅ Good practice: merges are on appropriate keys (trip_id, route_id), and types are cast explicitly.

In [ ]:
# trips.columns

In [ ]:
# routes.columns

In [ ]:
# routes.head()

In [ ]:
#This cell caused some issues in the merge because the df_with_trips and routes dfs did not agree on route_short_name, so we are going to take another approach
# #✅ Step 4b: Merge with routes.txt to Get Descriptive Info
# df_with_trips["route_id"] = df_with_trips["route_id"].astype(str)

# # Ensure consistent types
# trips["shape_id"] = trips["shape_id"].astype(str)
# routes["route_id"] = routes["route_id"].astype(str)

# # Take all of df_with_trips and add on data from routes if the route_id matches across both dataframes.
# df_full = df_with_trips.merge(routes, on="route_id", how="left")


In [23]:
#Let's avoid the route_..._x and route_..._y situation on the merge:
#✅ Step 4b: Merge with routes.txt to Get Descriptive Info
df_with_trips["route_id"] = df_with_trips["route_id"].astype(str)

# Ensure consistent types
trips["shape_id"] = trips["shape_id"].astype(str)
routes["route_id"] = routes["route_id"].astype(str)

# Assume 'df_with_trips' and 'routes' both contain 'route_id' and 'route_short_name'

# Step 1: Check 1:1 match on overlapping column
merged_check = df_with_trips[['route_id', 'route_short_name']].merge(
    routes[['route_id', 'route_short_name']],
    on='route_id',
    how='inner',
    suffixes=('_df_with_trips', '_routes')
)


In [ ]:
# merged_check.head()

In [ ]:
# merged_check.info()

In [24]:
##Note: This outputting an error is working as intended; it is showing the mismatches, just as an error instead of a standard output
# Step 2: Find any mismatches
mismatch = merged_check[merged_check['route_short_name_df_with_trips'] != merged_check['route_short_name_routes']]

if not mismatch.empty:
    raise ValueError(f"Mismatch found in route_short_name across dataframes:\n{mismatch}")

ValueError: Mismatch found in route_short_name across dataframes:
      route_id route_short_name_df_with_trips route_short_name_routes
10        4331                            766                     777
25        4319                             50                      36
53        4321                             10                       5
64        4330                            777                     766
68        4331                            766                     777
...        ...                            ...                     ...
84249     4331                            766                     777
84269     4330                            777                     766
84274     4331                            766                     777
84296     4330                            777                     766
84301     4331                            766                     777

[2198 rows x 3 columns]

In [25]:
#Okay... so there are some mismatches between the data and the static routes info; let's see what they are
mismatch_summary = (
    mismatch[['route_id', 'route_short_name_df_with_trips', 'route_short_name_routes']]
    .drop_duplicates()
    .sort_values('route_id')
)
display(mismatch_summary)


,route_id,route_short_name_df_with_trips,route_short_name_routes
4496,4301,5,10
14882,4305,141,140
3572,4306,140,141
25,4319,50,36
53,4321,10,5
64,4330,777,766
10,4331,766,777
2514,4333,11,8


#This is the stopping point for 250731-1200; pick up here

In [ ]:
# [k for k, v in globals().items() if str(type(v)) == "<class 'pandas.core.frame.DataFrame'>"]


In [ ]:
# routes.head()


In [ ]:
# routes.columns

In [26]:
# Rename to preserve both versions of route_short_name
routes_renamed = routes.rename(columns={'route_short_name': 'route_short_name_static'})

# Merge with df_with_trips, keeping the RT version as primary
merged = df_with_trips.merge(routes_renamed, on='route_id', how='left')


In [ ]:
# merged.columns

In [27]:
df_full = merged

In [ ]:
# df_full.describe(include='all')

In [ ]:
# #✅ Step 5b: Load shapes.txt
# shapes = pd.read_csv("/content/gtfs_static/shapes.txt", dtype={"shape_id": str})

In [ ]:
# shapes.describe(include='all')

In [ ]:
# shapes.head(1000)

In [28]:
#✅ Step 5b: Load shapes.txt
shapes = pd.read_csv("/content/gtfs_static/shapes.txt", dtype={"shape_id": str})

# Build LineStrings for each shape_id
shape_lines = {}
for shape_id, group in shapes.groupby("shape_id"):
    sorted_group = group.sort_values("shape_pt_sequence")
    coords = list(zip(sorted_group["shape_pt_lon"], sorted_group["shape_pt_lat"]))
    shape_lines[shape_id] = LineString(coords)

# Merge trips and routes to link shape_id to route_short_name
shape_route_map = (
    trips.merge(routes, on="route_id", how="left")
         .dropna(subset=["route_short_name"])
         .drop_duplicates(subset=["shape_id"])
         .set_index("shape_id")[["route_id", "route_short_name"]]
)

In [29]:
#✅ Step 6b: Create a data_quality Flag
def classify_row(row):
    if row["trip_id"] in invalid_trip_ids:
        return "invalid_trip_id"
    elif pd.isna(row["route_id"]):
        return "missing_route_id"
    elif pd.isna(row["route_long_name"]):
        return "missing_route_metadata"
    else:
        return "valid"

df_full["data_quality"] = df_full.apply(classify_row, axis=1)
print(df_full["data_quality"].value_counts())


data_quality
valid    84318
Name: count, dtype: int64


#####Possible Deprication

In [ ]:
#Not sure if this is needed
# #Load the trips, routes, shapes from Static data
# with zipfile.ZipFile(gtfs_zip_path, 'r') as z:
#     trips_df = pd.read_csv(z.open("trips.txt"))
#     routes_df = pd.read_csv(z.open("routes.txt"))
#     # Load GTFS shapes.txt into a DataFrame
#     shapes_df = pd.read_csv(z.open("shapes.txt"))

#Clean the RT data

In [30]:
#🔹 1.1 Filter for Valid Rows
df_valid = df_full[df_full["data_quality"] == "valid"].copy()

#🔹 1.2 Parse Timestamps
df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)

#🔹 1.3 Sort by Vehicle and Timestamp
df_valid = df_valid.sort_values(by=["vehicle_id", "timestamp"])

#🔹 1.4 Organize by Vehicle
#This creates a dictionary keyed by vehicle ID, each with a sorted DataFrame:
vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

#You can confirm how many distinct vehicles you’re tracking:
print("Vehicle count:", len(vehicle_groups))


Vehicle count: 72


In [ ]:
# df_full.shape

In [ ]:
# df_full.describe(include='all')

In [ ]:
# df_full.head()

In [ ]:
# df_valid.shape

In [ ]:
# df_valid.describe(include='all')

In [ ]:
# df_valid.head(1000)

#Anomly Detection

🔹 1. Detect Jumps and Gaps

Already implemented, but here’s the modular form:

In [ ]:
# # 🔹 1. Detect Jumps and Gaps
# def detect_jumps_and_gaps(df, params):
#     """
#     Detects jump or gap anomalies based on distance (meters) and time (seconds)
#     thresholds pulled from the anomaly_params dictionary.

#     Parameters:
#         df (pd.DataFrame): Filtered DataFrame for a single vehicle
#         params (dict): Anomaly detection thresholds from widget interface

#     Returns:
#         List[dict]: List of detected jump or gap anomalies
#     """
#     distance_threshold = params["JUMP_DISTANCE_METERS"]           # in meters
#     time_threshold = params["DISAPPEARANCE_TIME_THRESHOLD"]       # in seconds

#     anomalies = []
#     for i in range(1, len(df)):
#         row_prev, row_curr = df.iloc[i - 1], df.iloc[i]
#         time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()

#         distance = geodesic(
#             (row_prev["latitude"], row_prev["longitude"]),
#             (row_curr["latitude"], row_curr["longitude"])
#         ).meters

#         if time_diff > time_threshold or distance > distance_threshold:
#             anomalies.append({
#                 "vehicle_id": row_curr["vehicle_id"],
#                 "timestamp_prev": row_prev["timestamp"],
#                 "timestamp_curr": row_curr["timestamp"],
#                 "time_diff_sec": time_diff,
#                 "distance_m": distance,
#                 "is_gap": time_diff > time_threshold,
#                 "is_jump": distance > distance_threshold,
#                 "anomaly_type": "jump_or_gap"
#             })

#     return anomalies
# #Note: changed row["timestamp_collected"] to row["timestamp"] throughout, assuming that we've already converted timestamp_collected to UTC and assigned it to a new "timestamp" column at the start of the anomaly pipeline (which the existing pipeline does). This avoids inconsistency.

In [31]:
# 🔹 1. Detect Jumps and Gaps – WITHIN trip_id
def detect_jumps_and_gaps(df, params):
    """
     Detects jump or gap anomalies based on distance (meters) and time (seconds)
     thresholds pulled from the anomaly_params dictionary.

    Parameters:
        df (pd.DataFrame): Filtered DataFrame for a single vehicle, sorted by timestamp.
                           Must include 'trip_id', 'timestamp', 'latitude', 'longitude'.
        params (dict): Anomaly detection thresholds from widget interface.

    Returns:
        List[dict]: Detected jump or gap anomalies.
    """
    distance_threshold = params["JUMP_DISTANCE_METERS"]           # meters
    time_threshold = params["DISAPPEARANCE_TIME_THRESHOLD"]       # seconds

    anomalies = []
    for i in range(1, len(df)):
        row_prev, row_curr = df.iloc[i - 1], df.iloc[i]

        # 💡 Trip boundary check — skip if different trip
        if row_prev["trip_id"] != row_curr["trip_id"]:
            continue

        time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()

        distance = geodesic(
            (row_prev["latitude"], row_prev["longitude"]),
            (row_curr["latitude"], row_curr["longitude"])
        ).meters

        if time_diff > time_threshold or distance > distance_threshold:
            anomalies.append({
                "vehicle_id": row_curr["vehicle_id"],
                "trip_id": row_curr["trip_id"],  # optional but useful for tracing
                "timestamp_prev": row_prev["timestamp"],
                "timestamp_curr": row_curr["timestamp"],
                "time_diff_sec": time_diff,
                "distance_m": distance,
                "is_gap": time_diff > time_threshold,
                "is_jump": distance > distance_threshold,
                "anomaly_type": "jump_or_gap"
            })

    return anomalies


In [ ]:
# #2. Detect Stuck Vehicles
# def detect_stuck_vehicle(df, params):
#     """
#     Detects stuck vehicles using a rolling window of speed and spatial consistency,
#     segmented by trip_id to avoid cross-trip contamination.

#     Parameters:
#         df (pd.DataFrame): Must contain 'speed_mph', 'latitude', 'longitude', and 'trip_id'.
#         params (dict): Widget-defined anomaly thresholds.

#     Returns:
#         pd.DataFrame: Rows flagged as 'stuck_vehicle'.
#     """
#     speed_thresh = params.get("STUCK_SPEED_MPH", 1.0)   # Default: 1 mph
#     window = params.get("STUCK_WINDOW_SIZE", 4)         # Default: 4-frame window

#     stuck_flags = (
#         (df["speed_mph"].rolling(window).mean() < speed_thresh) &
#         (df["latitude"].diff().abs().rolling(window).mean() < 0.0001) &
#         (df["longitude"].diff().abs().rolling(window).mean() < 0.0001)
#     )

#     return df[stuck_flags.fillna(False)].assign(anomaly_type="stuck_vehicle")


In [32]:
# 🔹 2. Detect Stuck Vehicles
def detect_stuck_vehicle(df, params):
    """
    Detects stuck vehicles using a rolling window of speed and spatial consistency,
    segmented by trip_id to avoid cross-trip contamination.

    Parameters:
        df (pd.DataFrame): Must contain 'speed_mph', 'latitude', 'longitude', and 'trip_id'.
        params (dict): Widget-defined anomaly thresholds.

    Returns:
        pd.DataFrame: Flagged stuck vehicle anomalies.
    """
    speed_thresh = params.get("STUCK_SPEED_MPH", 1.0)   # mph
    window = params.get("STUCK_WINDOW_SIZE", 4)         # frames

    flagged_list = []

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()

        # Compute rolling conditions
        speed_ok = group["speed_mph"].rolling(window).mean() < speed_thresh
        lat_ok = group["latitude"].diff().abs().rolling(window).mean() < 0.0001
        lon_ok = group["longitude"].diff().abs().rolling(window).mean() < 0.0001

        stuck_flags = speed_ok & lat_ok & lon_ok

        flagged = group[stuck_flags.fillna(False)].copy()
        if not flagged.empty:
            flagged["anomaly_type"] = "stuck_vehicle"
            flagged_list.append(flagged)

    return pd.concat(flagged_list, ignore_index=True) if flagged_list else pd.DataFrame(columns=df.columns.tolist() + ["anomaly_type"])


In [ ]:
# # 🔹 3. Detect Impossible Speeds
# def detect_impossible_speeds(df, params):
#     """
#     Detects movement segments where computed speed exceeds threshold.

#     Parameters:
#         df (pd.DataFrame): Filtered or per-vehicle data
#         params (dict): Anomaly detection parameters from UI

#     Returns:
#         pd.DataFrame: Rows with flagged impossible-speed anomalies
#     """
#     # Convert mph to kph for geodesic-based calc
#     speed_limit_kph = params["SPEED_LIMIT_MPH"] * 1.60934

#     records = []
#     for i in range(1, len(df)):
#         row_prev, row_curr = df.iloc[i - 1], df.iloc[i]
#         time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()
#         if time_diff == 0:
#             continue

#         distance = geodesic(
#             (row_prev["latitude"], row_prev["longitude"]),
#             (row_curr["latitude"], row_curr["longitude"])
#         ).meters

#         speed_kph = (distance / time_diff) * 3.6  # m/s → km/h

#         if speed_kph > speed_limit_kph:
#             records.append({
#                 "vehicle_id": row_curr["vehicle_id"],
#                 "timestamp_curr": row_curr["timestamp"],
#                 "computed_speed_kph": speed_kph,
#                 "distance_m": distance,
#                 "anomaly_type": "impossible_speed"
#             })

#     return pd.DataFrame(records)


In [33]:
# 🔹 3. Detect Impossible Speeds
def detect_impossible_speeds(df, params):
    """
    Detects movement segments where computed speed exceeds threshold,
    segmented by trip_id to avoid cross-trip errors.

    Parameters:
        df (pd.DataFrame): Must contain timestamp, lat/lon, trip_id
        params (dict): Anomaly detection thresholds

    Returns:
        pd.DataFrame: Flagged rows with 'impossible_speed'
    """
    speed_limit_kph = params["SPEED_LIMIT_MPH"] * 1.60934
    output_rows = []

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()

        for i in range(1, len(group)):
            row_prev, row_curr = group.iloc[i - 1], group.iloc[i]
            time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()
            if time_diff <= 0:
                continue

            distance = geodesic(
                (row_prev["latitude"], row_prev["longitude"]),
                (row_curr["latitude"], row_curr["longitude"])
            ).meters

            speed_kph = (distance / time_diff) * 3.6

            if speed_kph > speed_limit_kph:
                output_rows.append({
                    "vehicle_id": row_curr["vehicle_id"],
                    "trip_id": trip_id,
                    "timestamp_curr": row_curr["timestamp"],
                    "computed_speed_kph": speed_kph,
                    "distance_m": distance,
                    "anomaly_type": "impossible_speed"
                })

    return pd.DataFrame(output_rows)


In [ ]:
# # 🔹 4. Detect Backtracking (Heading Reversal)
# def detect_backtracking(df, params):
#     """
#     Detects heading reversals suggesting backtracking behavior.

#     Parameters:
#         df (pd.DataFrame): Filtered or per-vehicle data
#         params (dict): Anomaly detection parameters from UI

#     Returns:
#         pd.DataFrame: Rows where heading reversed beyond threshold
#     """
#     reversal_thresh = params["REVERSAL_HEADING_THRESHOLD"]

#     if "heading" not in df.columns:
#         return pd.DataFrame()

#     heading_diff = df["heading"].diff().abs()
#     backtrack_flags = heading_diff.between(reversal_thresh, 200)

#     return df[backtrack_flags.fillna(False)].assign(anomaly_type="backtracking")


In [34]:
# 🔹 4. Detect Backtracking (Heading Reversal)
def detect_backtracking(df, params):
    """
    Detects heading reversals (backtracking) within trip boundaries only,
    with wraparound-safe angle diffing and traceability.

    Parameters:
        df (pd.DataFrame): Filtered per-vehicle data with heading + trip_id
        params (dict): Anomaly detection thresholds

    Returns:
        pd.DataFrame: Rows flagged as 'backtracking'
    """
    reversal_thresh = params.get("REVERSAL_HEADING_THRESHOLD", 120)
    output_frames = []

    if "heading" not in df.columns or "trip_id" not in df.columns:
        return pd.DataFrame()

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()

        # Compute heading difference with wraparound correction
        group["heading_prev"] = group["heading"].shift()
        group["heading_diff"] = (group["heading"] - group["heading_prev"]).abs()
        group["heading_diff"] = group["heading_diff"].apply(
            lambda x: min(x, 360 - x) if pd.notna(x) else x
        )

        # Identify backtracking
        backtrack_flags = group["heading_diff"].between(reversal_thresh, 200)

        flagged = group[backtrack_flags.fillna(False)].copy()
        flagged["anomaly_type"] = "backtracking"
        flagged["timestamp_prev"] = group["timestamp"].shift()

        output_frames.append(flagged)

    return pd.concat(output_frames, ignore_index=True) if output_frames else pd.DataFrame()



In [ ]:
# # 🔹 5. Detect Repeated Points
# def detect_repeated_points(df, params):
#     """
#     Detects repeated GPS coordinates (vehicle not moving).

#     Parameters:
#         df (pd.DataFrame): Filtered or per-vehicle snapshot data
#         params (dict): Anomaly detection parameters from UI

#     Returns:
#         pd.DataFrame: Rows flagged as having repeated lat/lon values
#     """
#     tolerance = abs(params.get("REPEATED_COORD_TOLERANCE", 1e-5))

#     if "latitude" not in df.columns or "longitude" not in df.columns:
#         return pd.DataFrame()

#     repeated = (
#         (df["latitude"].diff().abs() < tolerance) &
#         (df["longitude"].diff().abs() < tolerance)
#     )

#     return df[repeated.fillna(False)].assign(anomaly_type="repeated_points")


In [35]:
# 🔹 5. Detect Repeated Points
def detect_repeated_points(df, params):
    """
    Detects repeated GPS coordinates (vehicle not moving), scoped to each trip.

    Parameters:
        df (pd.DataFrame): Per-vehicle GPS snapshot data
        params (dict): UI-provided anomaly detection thresholds

    Returns:
        pd.DataFrame: Rows flagged as 'repeated_points'
    """
    tolerance = abs(params.get("REPEATED_COORD_TOLERANCE", 1e-5))
    output = []

    if "latitude" not in df.columns or "longitude" not in df.columns or "trip_id" not in df.columns:
        return pd.DataFrame()

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()
        lat_repeat = group["latitude"].diff().abs() < tolerance
        lon_repeat = group["longitude"].diff().abs() < tolerance
        repeated = lat_repeat & lon_repeat

        flagged = group[repeated.fillna(False)].copy()
        flagged["anomaly_type"] = "repeated_points"
        output.append(flagged)

    return pd.concat(output, ignore_index=True) if output else pd.DataFrame()


In [ ]:
# # 🔹 6. Detect Disappearance Without Return
# def detect_disappeared(df, snapshot_end_time, params):
#     """
#     Detects vehicles that have not reappeared by the end of the snapshot period.

#     Parameters:
#         df (pd.DataFrame): Data for a single vehicle
#         snapshot_end_time (datetime): End of data collection
#         params (dict): Anomaly detection thresholds from widget

#     Returns:
#         pd.DataFrame: Single-row disappearance anomaly, or empty DataFrame
#     """
#     if df.empty:
#         return pd.DataFrame()

#     last_seen = df["timestamp"].max()
#     time_gap_sec = (snapshot_end_time - last_seen).total_seconds()
#     disappearance_thresh = params["DISAPPEARANCE_TIME_THRESHOLD"]

#     if time_gap_sec > disappearance_thresh:
#         return pd.DataFrame([{
#             "vehicle_id": df["vehicle_id"].iloc[0],
#             "last_seen": last_seen,
#             "anomaly_type": "disappearance"
#         }])

#     return pd.DataFrame()


In [36]:
# 🔹 6. Detect Disappearance Without Return
def detect_disappeared(df, snapshot_end_time, params):
    """
    Detects disappearance per trip_id — if the trip ends and never resumes.

    Parameters:
        df (pd.DataFrame): Data for a single vehicle
        snapshot_end_time (datetime): End of snapshot
        params (dict): Anomaly config

    Returns:
        pd.DataFrame: One row per disappeared trip, if any
    """
    disappearance_thresh = params["DISAPPEARANCE_TIME_THRESHOLD"]
    output = []

    for trip_id, group in df.groupby("trip_id"):
        last_seen = group["timestamp"].max()
        time_gap_sec = (snapshot_end_time - last_seen).total_seconds()

        if time_gap_sec > disappearance_thresh:
            output.append({
                "vehicle_id": group["vehicle_id"].iloc[0],
                "trip_id": trip_id,
                "last_seen": last_seen,
                "anomaly_type": "disappearance"
            })

    return pd.DataFrame(output) if output else pd.DataFrame()


In [37]:
# 🔹 7. Detect Early Appearance
def detect_early_appearance(df, snapshot_start_time, params):
    """
    Detects vehicles that appear too early (likely pre-start ghost data).

    Parameters:
        df (pd.DataFrame): Data for a single vehicle
        snapshot_start_time (datetime): Start of capture window
        params (dict): Anomaly detection thresholds from widget

    Returns:
        pd.DataFrame: Single-row anomaly, or empty DataFrame
    """
    margin_seconds = params.get("EARLY_APPEARANCE_MARGIN_SEC", 30)
    first_seen = df["timestamp"].min()
    delta = (first_seen - snapshot_start_time).total_seconds()

    if delta < margin_seconds:
        return pd.DataFrame([{
            "vehicle_id": df["vehicle_id"].iloc[0],
            "first_seen": first_seen,
            "anomaly_type": "early_appearance"
        }])
    return pd.DataFrame()


In [ ]:
# # 🔹 8. Detect Off-Route Movement
# def detect_off_route(df_vehicle, shape_lines, params):
#     """
#     Flags GPS points that are farther than OFF_ROUTE_BUFFER_M from the expected route shape.

#     Parameters:
#         df_vehicle (pd.DataFrame): All points for a single vehicle
#         shape_lines (dict): Dictionary of LineStrings per shape_id
#         params (dict): Thresholds including OFF_ROUTE_BUFFER_M

#     Returns:
#         pd.DataFrame: Off-route GPS points
#     """
#     buffer_m = params.get("OFF_ROUTE_BUFFER_M", 50)
#     records = []

#     for _, row in df_vehicle.iterrows():
#         shape_id = str(row.get("shape_id"))
#         if shape_id not in shape_lines:
#             continue  # shape not available

#         route_line = shape_lines[shape_id]
#         vehicle_point = Point(row["longitude"], row["latitude"])

#         # Project point onto shape line and compute geodesic distance
#         closest_point = route_line.interpolate(route_line.project(vehicle_point))
#         dist_m = geodesic(
#             (row["latitude"], row["longitude"]),
#             (closest_point.y, closest_point.x)
#         ).meters

#         if dist_m > buffer_m:
#             records.append({
#                 "vehicle_id": row["vehicle_id"],
#                 "timestamp": row["timestamp"],
#                 "route_short_name": row.get("route_short_name"),
#                 "distance_from_route_m": dist_m,
#                 "latitude": row["latitude"],
#                 "longitude": row["longitude"],
#                 "shape_id": shape_id,
#                 "anomaly_type": "off_route"
#             })

#     return pd.DataFrame(records)


In [38]:
# 🔹 8. Detect Off-Route Movement
def detect_off_route(df_vehicle, shape_lines, params):
    """
    Detect off-route anomalies using trip-level shape validation.

    Parameters:
        df_vehicle (pd.DataFrame): All rows for a single vehicle.
        shape_lines (dict): shape_id → LineString
        params (dict): Dictionary with OFF_ROUTE_BUFFER_M

    Returns:
        pd.DataFrame: Flagged off-route points
    """
    buffer_m = params.get("OFF_ROUTE_BUFFER_M", 50)
    records = []

    for trip_id, trip_df in df_vehicle.groupby("trip_id"):
        # Check if all rows share the same shape_id
        shape_ids = trip_df["shape_id"].dropna().unique()

        if len(shape_ids) != 1:
            # Ambiguous shape_id — skip this trip to avoid misflagging
            continue

        shape_id = str(shape_ids[0])
        if shape_id not in shape_lines:
            continue

        route_line = shape_lines[shape_id]

        for _, row in trip_df.iterrows():
            vehicle_point = Point(row["longitude"], row["latitude"])
            projected = route_line.interpolate(route_line.project(vehicle_point))
            dist_m = geodesic(
                (row["latitude"], row["longitude"]),
                (projected.y, projected.x)
            ).meters

            if dist_m > buffer_m:
                records.append({
                    "vehicle_id": row["vehicle_id"],
                    "trip_id": trip_id,
                    "timestamp": row["timestamp"],
                    "route_short_name": row.get("route_short_name"),
                    "distance_from_route_m": dist_m,
                    "latitude": row["latitude"],
                    "longitude": row["longitude"],
                    "shape_id": shape_id,
                    "anomaly_type": "off_route"
                })

    return pd.DataFrame(records)

# Note: ✅ Key Protections Added
# Fix	Explanation
# groupby("trip_id")	Ensures anomalies only detected within a single trip context
# unique shape_id check	Filters out data where shape mapping is ambiguous
# skip trip if shape_id missing or mismatched	Prevents false positives from misassigned shapes
# Includes trip_id in output	Improves downstream traceability and debugging

In [39]:
# Define options
detector_options = [
    ("Jumps & Gaps", "jumpgap"),
    ("Stuck Vehicles", "stuck"),
    ("Impossible Speeds", "speed"),
    ("Backtracking", "backtrack"),
    ("Repeated Points", "repeated"),
    ("Disappearance", "disappear"),
    ("Early Appearance", "early"),
    ("Off-Route", "offroute")
]

# Create selector widget
detector_selector = widgets.SelectMultiple(
    options=detector_options,
    value=[k for _, k in detector_options],  # default: all selected
    description="Detectors:",
    layout=widgets.Layout(width="60%")
)

# Create button
confirm_button = widgets.Button(
    description="Confirm Selection",
    button_style='primary',
    icon='check'
)

# Output area
selection_output = widgets.Output()

# Define callback
def on_confirm_clicked(b):
    with selection_output:
        clear_output()
        selected = list(detector_selector.value)
        print("✅ Selected anomaly detectors:")
        for detector in selected:
            print(f" - {detector}")
        # Optionally set a global variable for later use:
        global selected_detectors
        selected_detectors = selected

# Link button to callback
confirm_button.on_click(on_confirm_clicked)

# Display widgets
display(widgets.VBox([detector_selector, confirm_button, selection_output]))


In [ ]:
# # I believe we can skip step 1 because it was already done earlier
# # # Step 1: Normalize timestamp and rebuild vehicle groups
# # df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)
# # vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

# # Step 2: Detect each anomaly type using widget-supplied parameters
# jumpgap_records = []
# stuck_records = []
# speed_records = []
# backtrack_records = []
# repeated_records = []
# disappear_records = []
# early_records = []
# offroute_records = []

# snapshot_start = df_valid["timestamp"].min()
# snapshot_end = df_valid["timestamp"].max()

# for vehicle_id, df_vehicle in vehicle_groups.items():
#     df_vehicle = df_vehicle.sort_values("timestamp").reset_index(drop=True)

#     # 1. Jumps and Gaps
#     jumpgap_records.extend(detect_jumps_and_gaps(df_vehicle, params=anomaly_params))

#     # 2. Stuck Vehicles
#     stuck = detect_stuck_vehicle(df_vehicle, params=anomaly_params)
#     if not stuck.empty:
#         stuck_records.append(stuck)

#     # 3. Impossible Speeds
#     speed = detect_impossible_speeds(df_vehicle, params=anomaly_params)
#     if not speed.empty:
#         speed_records.append(speed)

#     # 4. Backtracking
#     backtrack = detect_backtracking(df_vehicle, params=anomaly_params)
#     if not backtrack.empty:
#         backtrack_records.append(backtrack)

#     # 5. Repeated Points
#     repeat = detect_repeated_points(df_vehicle, params=anomaly_params)
#     if not repeat.empty:
#         repeated_records.append(repeat)

#     # 6. Disappearance
#     disappear = detect_disappeared(df_vehicle, snapshot_end_time=snapshot_end, params=anomaly_params)
#     if not disappear.empty:
#         disappear_records.append(disappear)

#     # 7. Early Appearance
#     early = detect_early_appearance(df_vehicle, snapshot_start_time=snapshot_start, params=anomaly_params)
#     if not early.empty:
#         early_records.append(early)

#     # 8. Off-Route
#     offroute = detect_off_route(df_vehicle, shape_lines=shape_lines, params=anomaly_params)
#     if not offroute.empty:
#         offroute_records.append(offroute)

# # Step 3: Combine to DataFrames
# df_anomalies_jumpgap   = pd.DataFrame(jumpgap_records)
# df_anomalies_stuck     = pd.concat(stuck_records, ignore_index=True)     if stuck_records     else pd.DataFrame()
# df_anomalies_speed     = pd.concat(speed_records, ignore_index=True)     if speed_records     else pd.DataFrame()
# df_anomalies_backtrack = pd.concat(backtrack_records, ignore_index=True) if backtrack_records else pd.DataFrame()
# df_anomalies_repeated  = pd.concat(repeated_records, ignore_index=True)  if repeated_records  else pd.DataFrame()
# df_anomalies_disappear = pd.concat(disappear_records, ignore_index=True) if disappear_records else pd.DataFrame()
# df_anomalies_early     = pd.concat(early_records, ignore_index=True)     if early_records     else pd.DataFrame()
# df_anomalies_offroute  = pd.concat(offroute_records, ignore_index=True)  if offroute_records  else pd.DataFrame()

# # Step 4: Combine all anomalies into a single DataFrame
# anomaly_frames = [
#     df_anomalies_jumpgap,
#     df_anomalies_stuck,
#     df_anomalies_speed,
#     df_anomalies_backtrack,
#     df_anomalies_repeated,
#     df_anomalies_disappear,
#     df_anomalies_early,
#     df_anomalies_offroute
# ]

# anomaly_frames = [df for df in anomaly_frames if 'anomaly_type' in df.columns and not df.empty]
# df_anomalies_full = pd.concat(anomaly_frames, ignore_index=True)

# # Report
# print("✅ Unified anomaly count:", len(df_anomalies_full))
# display(df_anomalies_full["anomaly_type"].value_counts())


In [41]:
#Debugging the anomaly index dupe issue
for i, df in enumerate(anomaly_frames):
    dupes = df.columns[df.columns.duplicated()].tolist()
    if dupes:
        print(f"⚠️ Duplicate columns in anomaly_frames[{i}]: {dupes}")


⚠️ Duplicate columns in anomaly_frames[1]: ['next_stop_id']
⚠️ Duplicate columns in anomaly_frames[3]: ['next_stop_id']
⚠️ Duplicate columns in anomaly_frames[4]: ['next_stop_id']


In [42]:
# Selected detectors from UI
selected_detectors = list(detector_selector.value)

# Reset result containers
jumpgap_records = []
stuck_records = []
speed_records = []
backtrack_records = []
repeated_records = []
disappear_records = []
early_records = []
offroute_records = []

snapshot_start = df_valid["timestamp"].min()
snapshot_end = df_valid["timestamp"].max()

# Run detectors conditionally
for vehicle_id, df_vehicle in vehicle_groups.items():
    df_vehicle = df_vehicle.sort_values("timestamp").reset_index(drop=True)

    if "jumpgap" in selected_detectors:
        jumpgap_records.extend(detect_jumps_and_gaps(df_vehicle, params=anomaly_params))

    if "stuck" in selected_detectors:
        stuck = detect_stuck_vehicle(df_vehicle, params=anomaly_params)
        if not stuck.empty:
            stuck_records.append(stuck)

    if "speed" in selected_detectors:
        speed = detect_impossible_speeds(df_vehicle, params=anomaly_params)
        if not speed.empty:
            speed_records.append(speed)

    if "backtrack" in selected_detectors:
        backtrack = detect_backtracking(df_vehicle, params=anomaly_params)
        if not backtrack.empty:
            backtrack_records.append(backtrack)

    if "repeated" in selected_detectors:
        repeat = detect_repeated_points(df_vehicle, params=anomaly_params)
        if not repeat.empty:
            repeated_records.append(repeat)

    if "disappear" in selected_detectors:
        disappear = detect_disappeared(df_vehicle, snapshot_end_time=snapshot_end, params=anomaly_params)
        if not disappear.empty:
            disappear_records.append(disappear)

    if "early" in selected_detectors:
        early = detect_early_appearance(df_vehicle, snapshot_start_time=snapshot_start, params=anomaly_params)
        if not early.empty:
            early_records.append(early)

    if "offroute" in selected_detectors:
        offroute = detect_off_route(df_vehicle, shape_lines=shape_lines, params=anomaly_params)
        if not offroute.empty:
            offroute_records.append(offroute)

# Step 3: Combine to DataFrames
df_anomalies_jumpgap   = pd.DataFrame(jumpgap_records)
df_anomalies_stuck     = pd.concat(stuck_records, ignore_index=True)     if stuck_records     else pd.DataFrame()
df_anomalies_speed     = pd.concat(speed_records, ignore_index=True)     if speed_records     else pd.DataFrame()
df_anomalies_backtrack = pd.concat(backtrack_records, ignore_index=True) if backtrack_records else pd.DataFrame()
df_anomalies_repeated  = pd.concat(repeated_records, ignore_index=True)  if repeated_records  else pd.DataFrame()
df_anomalies_disappear = pd.concat(disappear_records, ignore_index=True) if disappear_records else pd.DataFrame()
df_anomalies_early     = pd.concat(early_records, ignore_index=True)     if early_records     else pd.DataFrame()
df_anomalies_offroute  = pd.concat(offroute_records, ignore_index=True)  if offroute_records  else pd.DataFrame()

# Step 4: Combine all selected anomalies
anomaly_frames = []
if "jumpgap" in selected_detectors:   anomaly_frames.append(df_anomalies_jumpgap)
if "stuck" in selected_detectors:     anomaly_frames.append(df_anomalies_stuck)
if "speed" in selected_detectors:     anomaly_frames.append(df_anomalies_speed)
if "backtrack" in selected_detectors: anomaly_frames.append(df_anomalies_backtrack)
if "repeated" in selected_detectors:  anomaly_frames.append(df_anomalies_repeated)
if "disappear" in selected_detectors: anomaly_frames.append(df_anomalies_disappear)
if "early" in selected_detectors:     anomaly_frames.append(df_anomalies_early)
if "offroute" in selected_detectors:  anomaly_frames.append(df_anomalies_offroute)

anomaly_frames = [df for df in anomaly_frames if 'anomaly_type' in df.columns and not df.empty]

#New *v9.2 code to try to trim off duplicate columns
# Step 5: Clean columns, concatenate frames, and inspect duplicates
anomaly_frames_cleaned = []
for i, df in enumerate(anomaly_frames):
    if not df.columns.is_unique:
        dup_cols = df.columns[df.columns.duplicated()].tolist()
        print(f"⚠️ Cleaning duplicate columns in anomaly_frames[{i}]: {dup_cols}")
        df = df.loc[:, ~df.columns.duplicated()]  # Drop duplicate columns, keep first
    anomaly_frames_cleaned.append(df)

# Concatenate all cleaned frames
df_anomalies_full = pd.concat(anomaly_frames_cleaned, ignore_index=True) if anomaly_frames_cleaned else pd.DataFrame()

# Inspect and drop row duplicates
if not df_anomalies_full.empty:
    df_anomalies_duplicates = df_anomalies_full[df_anomalies_full.duplicated(keep='first')]
    print("🔎 Duplicate rows to be dropped:", len(df_anomalies_duplicates))
    display(df_anomalies_duplicates.head())
    df_anomalies_full.drop_duplicates(inplace=True)
else:
    df_anomalies_duplicates = pd.DataFrame()  # Fallback if no data

#Previous code before the switch above
#df_anomalies_full = pd.concat(anomaly_frames, ignore_index=True) if anomaly_frames else pd.DataFrame()

# Report
print("Anomaly count (pre-dedup):", len(df_anomalies_full))
df_anomalies_full.drop_duplicates(inplace=True)
print("Anomaly count (post-dedup):", len(df_anomalies_full))

print("✅ Unified anomaly count:", len(df_anomalies_full))
if not df_anomalies_full.empty:
    display(df_anomalies_full["anomaly_type"].value_counts())
else:
    print("⚠️ No anomalies detected.")


⚠️ Cleaning duplicate columns in anomaly_frames[1]: ['next_stop_id']
⚠️ Cleaning duplicate columns in anomaly_frames[3]: ['next_stop_id']
⚠️ Cleaning duplicate columns in anomaly_frames[4]: ['next_stop_id']
🔎 Duplicate rows to be dropped: 23084


,vehicle_id,trip_id,timestamp_prev,timestamp_curr,time_diff_sec,distance_m,is_gap,is_jump,anomaly_type,id,...,route_color,route_text_color,data_quality,timestamp,computed_speed_kph,heading_prev,heading_diff,last_seen,first_seen,distance_from_route_m
30009,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
30130,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
30199,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
30288,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
30424,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN


Anomaly count (pre-dedup): 101632
Anomaly count (post-dedup): 101632
✅ Unified anomaly count: 101632


,count
anomaly_type,
repeated_points,63374
stuck_vehicle,13737
jump_or_gap,11973
impossible_speed,4205
off_route,3768
backtracking,945
disappearance,742
early_appearance,25


In [43]:
df_anomalies_full.describe(include='all')

,vehicle_id,trip_id,timestamp_prev,timestamp_curr,time_diff_sec,distance_m,is_gap,is_jump,anomaly_type,id,...,route_color,route_text_color,data_quality,timestamp,computed_speed_kph,heading_prev,heading_diff,last_seen,first_seen,distance_from_route_m
count,98769,98744,15780,16178,11973.000000,16178.000000,11973,11973,98769,7.805600e+04,...,78056,78056,78056,81824,4205.000000,945.000000,945.000000,742,25,3768.000000
unique,72,805,NaN,NaN,NaN,NaN,2,2,8,NaN,...,21,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,383,601256,NaN,NaN,NaN,NaN,False,True,repeated_points,NaN,...,00DEC8,FFFFFF,valid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,4445,2688,NaN,NaN,NaN,NaN,11971,11971,63374,NaN,...,8830,47221,78056,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,2025-08-04 16:18:15.407204608+00:00,2025-08-04 17:04:09.228748544+00:00,30.174548,989.241020,NaN,NaN,NaN,3.615106e+06,...,NaN,NaN,NaN,2025-08-04 17:01:50.344931584+00:00,149.455764,163.314286,159.119577,2025-08-04 17:24:55.124378368+00:00,2025-08-04 00:00:09.122007040+00:00,804.879744
min,NaN,NaN,2025-08-04 00:00:09.122007+00:00,2025-08-04 00:02:09.221975+00:00,27.099802,76.362764,NaN,NaN,NaN,2.908529e+06,...,NaN,NaN,NaN,2025-08-04 00:00:09.122007+00:00,112.656385,0.000000,120.000000,2025-08-04 00:05:08.721672+00:00,2025-08-04 00:00:09.122007+00:00,50.013339
25%,NaN,NaN,2025-08-04 13:46:34.922564096+00:00,2025-08-04 14:32:09.825888+00:00,29.599582,717.205157,NaN,NaN,NaN,3.515370e+06,...,NaN,NaN,NaN,2025-08-04 14:39:40.821731072+00:00,123.829710,76.000000,144.000000,2025-08-04 15:04:12.172080384+00:00,2025-08-04 00:00:09.122007040+00:00,64.606324
50%,NaN,NaN,2025-08-04 17:13:05.421704960+00:00,2025-08-04 17:34:37.021687040+00:00,30.000227,949.682904,NaN,NaN,NaN,3.643014e+06,...,NaN,NaN,NaN,2025-08-04 17:47:36.822228992+00:00,138.367339,174.000000,166.000000,2025-08-04 17:58:37.721721088+00:00,2025-08-04 00:00:09.122007040+00:00,135.190045
75%,NaN,NaN,2025-08-04 20:34:31.323069952+00:00,2025-08-04 20:41:00.921515008+00:00,30.404542,1161.590420,NaN,NaN,NaN,3.767199e+06,...,NaN,NaN,NaN,2025-08-04 20:51:00.223159040+00:00,162.041652,264.000000,175.000000,2025-08-04 20:51:30.421774336+00:00,2025-08-04 00:00:09.122007040+00:00,370.982145
max,NaN,NaN,2025-08-04 23:59:05.421088+00:00,2025-08-04 23:59:35.721861+00:00,689.701777,3506.450308,NaN,NaN,NaN,3.896207e+06,...,NaN,NaN,NaN,2025-08-04 23:59:35.721861+00:00,412.521206,359.000000,180.000000,2025-08-04 23:54:35.420812+00:00,2025-08-04 00:00:09.122007+00:00,13100.052039


In [44]:
df_anomalies_full.head(100)

,vehicle_id,trip_id,timestamp_prev,timestamp_curr,time_diff_sec,distance_m,is_gap,is_jump,anomaly_type,id,...,route_color,route_text_color,data_quality,timestamp,computed_speed_kph,heading_prev,heading_diff,last_seen,first_seen,distance_from_route_m
0,1902,602646,2025-08-04 00:01:39.522324+00:00,2025-08-04 00:02:09.221975+00:00,29.699651,804.604865,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
1,1902,602646,2025-08-04 00:05:08.721672+00:00,2025-08-04 00:05:39.321847+00:00,30.600175,739.278374,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
2,1902,602646,2025-08-04 00:07:08.921636+00:00,2025-08-04 00:07:39.422088+00:00,30.500452,616.148094,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
3,1902,602646,2025-08-04 00:08:40.421770+00:00,2025-08-04 00:09:10.422470+00:00,30.000700,590.497974,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
4,1902,602646,2025-08-04 00:10:39.520785+00:00,2025-08-04 00:11:09.421332+00:00,29.900547,561.596267,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1902,602935,2025-08-04 14:47:10.121920+00:00,2025-08-04 14:47:39.421560+00:00,29.299640,918.630264,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
96,1902,602935,2025-08-04 15:03:12.322370+00:00,2025-08-04 15:03:42.321936+00:00,29.999566,1131.058916,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
97,1902,602935,2025-08-04 15:07:11.721211+00:00,2025-08-04 15:07:42.321270+00:00,30.600059,976.120559,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
98,1902,602935,2025-08-04 15:08:42.521873+00:00,2025-08-04 15:09:12.821041+00:00,30.299168,1090.453224,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN


In [ ]:
# print("Total rows:", len(df_anomalies_full))
# print("Unique rows:", len(df_anomalies_full.drop_duplicates()))
# print("Duplicate rows:", df_anomalies_full.duplicated().sum())


####Merge check
2. Shape-Based Merge into df_anomalies_offroute
- df_anomalies_offroute = df_anomalies_offroute.merge(
    shape_route_map, on="shape_id", how="left", suffixes=("", "_from_map")
)

🔍 Key issue: This assumes that df_anomalies_offroute has an accurate shape_id per row.

But this is not always valid, because:

- Anomalies are grouped by vehicle_id and timestamp.

- There's no explicit trip_id or shape_id preserved per anomaly.

- If a single vehicle changes shapes mid-route, this could assign the wrong route.

In [45]:
shape_route_map.head()

,route_id,route_short_name
shape_id,,
17128,4300,1
17129,4300,1
17130,4301,10
17131,4301,10
17132,4301,10


In [46]:
shape_route_map.info()

<class 'pandas.core.frame.DataFrame'>
Index: 134 entries, 17128 to 17253
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   route_id          134 non-null    object
 1   route_short_name  134 non-null    object
dtypes: object(2)
memory usage: 3.1+ KB


In [47]:
df_anomalies_offroute.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3768 entries, 0 to 3767
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   vehicle_id             3768 non-null   object             
 1   trip_id                3768 non-null   object             
 2   timestamp              3768 non-null   datetime64[ns, UTC]
 3   route_short_name       3768 non-null   object             
 4   distance_from_route_m  3768 non-null   float64            
 5   latitude               3768 non-null   float64            
 6   longitude              3768 non-null   float64            
 7   shape_id               3768 non-null   object             
 8   anomaly_type           3768 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(3), object(5)
memory usage: 265.1+ KB


In [89]:
df_anomalies_offroute['shape_id'].value_counts()

,count
shape_id,
17213,789
17211,442
17143,293
17209,247
17214,218
17212,167
17161,164
17208,142
17152,106


In [49]:
# Ensure shape_id is a column (not index)
shape_route_map = shape_route_map.reset_index()

# Ensure both shape_id columns are string type before merge
df_anomalies_offroute["shape_id"] = df_anomalies_offroute["shape_id"].astype(str)
shape_route_map["shape_id"] = shape_route_map["shape_id"].astype(str)

# Join to enrich anomalies with route info
df_anomalies_offroute = (
    df_anomalies_offroute
    .merge(shape_route_map, on="shape_id", how="left", suffixes=("", "_from_map"))
)

# # Patch missing route names
# df_anomalies_offroute["route_short_name"] = (
#     df_anomalies_offroute["route_short_name"]
#     .fillna(df_anomalies_offroute["route_short_name_from_map"])
# )

# # Clean up
# df_anomalies_offroute = df_anomalies_offroute.drop(columns=["route_short_name_from_map"])


In [50]:
df_anomalies_offroute.describe(include='all')

,vehicle_id,trip_id,timestamp,route_short_name,distance_from_route_m,latitude,longitude,shape_id,anomaly_type,route_id,route_short_name_from_map
count,3768,3768,3768,3768,3768.000000,3768.000000,3768.000000,3768,3768,3768,3768
unique,70,335,NaN,22,NaN,NaN,NaN,50,1,22,22
top,1927,600883,NaN,766,NaN,NaN,NaN,17213,off_route,4331,777
freq,206,91,NaN,826,NaN,NaN,NaN,789,3768,1007,1007
mean,NaN,NaN,2025-08-04 14:12:56.305052160+00:00,NaN,804.879744,35.085098,-106.657395,NaN,NaN,NaN,NaN
min,NaN,NaN,2025-08-04 00:00:09.122007+00:00,NaN,50.013339,35.011017,-106.753163,NaN,NaN,NaN,NaN
25%,NaN,NaN,2025-08-04 12:16:03.321694976+00:00,NaN,64.606324,35.078789,-106.725784,NaN,NaN,NaN,NaN
50%,NaN,NaN,2025-08-04 16:29:35.022124032+00:00,NaN,135.190045,35.079587,-106.651249,NaN,NaN,NaN,NaN
75%,NaN,NaN,2025-08-04 20:18:30.622779904+00:00,NaN,370.982145,35.086432,-106.621682,NaN,NaN,NaN,NaN
max,NaN,NaN,2025-08-04 23:59:35.721861+00:00,NaN,13100.052039,35.198926,-106.496217,NaN,NaN,NaN,NaN


In [51]:
df_anomalies_offroute.isna().sum()

,0
vehicle_id,0
trip_id,0
timestamp,0
route_short_name,0
distance_from_route_m,0
latitude,0
longitude,0
shape_id,0
anomaly_type,0
route_id,0


In [52]:
#Sanity Check
missing_routes = df_anomalies_offroute["route_short_name"].isnull().sum()
print(f"Remaining anomalies with missing route_short_name: {missing_routes}")
#If this prints 0, you’ve successfully patched all entries.

if missing_routes > 0:
    print("Sample of anomalies with missing route_short_name:")
    display(df_anomalies_offroute[df_anomalies_offroute["route_short_name"].isnull()].head())


Remaining anomalies with missing route_short_name: 0


In [53]:
df_full.isna().sum()

,0
id,0
snapshot_id,0
msg_time,0
timestamp_collected,0
vehicle_id,0
longitude,0
latitude,0
heading,0
speed_mph,0
route_short_name,0


In [54]:
df_full.columns

Index(['id', 'snapshot_id', 'msg_time', 'timestamp_collected', 'vehicle_id',
       'longitude', 'latitude', 'heading', 'speed_mph', 'route_short_name',
       'trip_id', 'next_stop_id', 'next_stop_id', 'next_stop_name',
       'next_stop_sched_time', 'route_id', 'service_id', 'trip_headsign',
       'trip_short_name', 'direction_id', 'block_id', 'shape_id',
       'wheelchair_accessible', 'bikes_allowed', 'agency_id',
       'route_short_name_static', 'route_long_name', 'route_desc',
       'route_type', 'route_url', 'route_color', 'route_text_color',
       'data_quality'],
      dtype='object')

#####Debugging note: I believe the cell below may be creating some problems

In [55]:
df_full['data_quality'].value_counts()

,count
data_quality,
valid,84318


In [56]:
#🔹 1.1 Filter for Valid Rows
df_valid = df_full[df_full["data_quality"] == "valid"].copy()

#🔹 1.2 Parse Timestamps
df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)

#🔹 1.3 Sort by Vehicle and Timestamp
df_valid = df_valid.sort_values(by=["vehicle_id", "timestamp"])

#🔹 1.4 Organize by Vehicle
#This creates a dictionary keyed by vehicle ID, each with a sorted DataFrame:
vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

#You can confirm how many distinct vehicles you’re tracking:
print("Vehicle count:", len(vehicle_groups))


Vehicle count: 72


#Summary Stats by Route/Anomaly

####Debugging check: possible problem here with route_lookup

In [ ]:
# df_valid.head(20)

In [ ]:
# df_valid.columns

In [ ]:
# routes.columns

In [ ]:
# trips.columns

In [57]:
# Build trip-to-route lookup from GTFS static data
trip_to_route_lookup = trips[["trip_id", "route_id"]].merge(
    routes[["route_id", "route_short_name"]], on="route_id", how="left"
)

In [58]:
trip_to_route_lookup.columns

Index(['trip_id', 'route_id', 'route_short_name'], dtype='object')

In [59]:
trip_to_route_lookup['route_short_name'].value_counts()

,count
route_short_name,
766,430
777,404
66,396
141,346
11,242
5,220
10,214
8,214
157,210


In [60]:
trip_to_route_lookup['route_id'].value_counts()

,count
route_id,
4330,430
4331,404
4328,396
4306,346
4340,242
4321,220
4301,214
4333,214
4308,210


In [61]:
# Build trip-to-route lookup from GTFS static data
trip_to_route_lookup = trips[["trip_id", "route_id"]].merge(
    routes[["route_id", "route_short_name"]], on="route_id", how="left"
)

# Ensure trip_id types match
df_valid["trip_id"] = df_valid["trip_id"].astype(str)
trip_to_route_lookup["trip_id"] = trip_to_route_lookup["trip_id"].astype(str)

#I believe this may have been resolved by splitting route_short_name into x and y
# # Drop route_short_name from df_valid if it exists to avoid merge collision
# if "route_short_name" in df_valid.columns:
#     df_valid = df_valid.drop(columns=["route_short_name"])

trip_to_route_lookup["route_id_trips"] = trip_to_route_lookup["route_id"].copy()
trip_to_route_lookup["route_short_name_trips"] = trip_to_route_lookup["route_short_name"].copy()


# Merge to patch in route_short_name
df_valid = df_valid.merge(trip_to_route_lookup[["trip_id", "route_id_trips", "route_short_name_trips"]], on="trip_id", how="left")


In [62]:
df_valid.columns

Index(['id', 'snapshot_id', 'msg_time', 'timestamp_collected', 'vehicle_id',
       'longitude', 'latitude', 'heading', 'speed_mph', 'route_short_name',
       'trip_id', 'next_stop_id', 'next_stop_id', 'next_stop_name',
       'next_stop_sched_time', 'route_id', 'service_id', 'trip_headsign',
       'trip_short_name', 'direction_id', 'block_id', 'shape_id',
       'wheelchair_accessible', 'bikes_allowed', 'agency_id',
       'route_short_name_static', 'route_long_name', 'route_desc',
       'route_type', 'route_url', 'route_color', 'route_text_color',
       'data_quality', 'timestamp', 'route_id_trips',
       'route_short_name_trips'],
      dtype='object')

In [63]:
df_valid.head(10)

,id,snapshot_id,msg_time,timestamp_collected,vehicle_id,longitude,latitude,heading,speed_mph,route_short_name,...,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,data_quality,timestamp,route_id_trips,route_short_name_trips
0,2908203,5722,2025-08-05 17:59:39,2025-08-04 00:00:09.122007+00:00,1902,-106.609642,35.080502,279.0,0.0,766,...,ART Red Line,NaN,3,NaN,EC0000,FFFFFF,valid,2025-08-04 00:00:09.122007+00:00,4330,766
1,2908547,5723,2025-08-05 17:59:39,2025-08-04 00:00:39.422664+00:00,1902,-106.609642,35.080502,279.0,0.0,766,...,ART Red Line,NaN,3,NaN,EC0000,FFFFFF,valid,2025-08-04 00:00:39.422664+00:00,4330,766
2,2908891,5724,2025-08-05 17:59:39,2025-08-04 00:01:08.822636+00:00,1902,-106.609642,35.080502,279.0,0.0,766,...,ART Red Line,NaN,3,NaN,EC0000,FFFFFF,valid,2025-08-04 00:01:08.822636+00:00,4330,766
3,2909235,5725,2025-08-05 17:59:39,2025-08-04 00:01:39.522324+00:00,1902,-106.609642,35.080502,279.0,0.0,766,...,ART Red Line,NaN,3,NaN,EC0000,FFFFFF,valid,2025-08-04 00:01:39.522324+00:00,4330,766
4,2909579,5726,2025-08-05 18:01:39,2025-08-04 00:02:09.221975+00:00,1902,-106.618444,35.081004,273.0,17.0,766,...,ART Red Line,NaN,3,NaN,EC0000,FFFFFF,valid,2025-08-04 00:02:09.221975+00:00,4330,766
5,2909923,5727,2025-08-05 18:01:39,2025-08-04 00:02:39.322149+00:00,1902,-106.618444,35.081004,273.0,17.0,766,...,ART Red Line,NaN,3,NaN,EC0000,FFFFFF,valid,2025-08-04 00:02:39.322149+00:00,4330,766
6,2910267,5728,2025-08-05 18:01:39,2025-08-04 00:03:09.521555+00:00,1902,-106.618444,35.081004,273.0,17.0,766,...,ART Red Line,NaN,3,NaN,EC0000,FFFFFF,valid,2025-08-04 00:03:09.521555+00:00,4330,766
7,2910611,5729,2025-08-05 18:03:07,2025-08-04 00:03:39.622148+00:00,1902,-106.621773,35.080993,270.0,0.0,766,...,ART Red Line,NaN,3,NaN,EC0000,FFFFFF,valid,2025-08-04 00:03:39.622148+00:00,4330,766
8,2910955,5730,2025-08-05 18:03:07,2025-08-04 00:04:09.322325+00:00,1902,-106.621773,35.080993,270.0,0.0,766,...,ART Red Line,NaN,3,NaN,EC0000,FFFFFF,valid,2025-08-04 00:04:09.322325+00:00,4330,766
9,2911299,5731,2025-08-05 18:03:07,2025-08-04 00:04:39.221835+00:00,1902,-106.621773,35.080993,270.0,0.0,766,...,ART Red Line,NaN,3,NaN,EC0000,FFFFFF,valid,2025-08-04 00:04:39.221835+00:00,4330,766


In [64]:
#Step 1: Build a lookup table
route_lookup = df_valid[["vehicle_id", "timestamp", "route_short_name"]].copy()
route_lookup["timestamp"] = pd.to_datetime(route_lookup["timestamp"], utc=True)


In [65]:
df_anomalies_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101632 entries, 0 to 124715
Data columns (total 46 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   vehicle_id               98769 non-null  object             
 1   trip_id                  98744 non-null  object             
 2   timestamp_prev           15780 non-null  datetime64[ns, UTC]
 3   timestamp_curr           16178 non-null  datetime64[ns, UTC]
 4   time_diff_sec            11973 non-null  float64            
 5   distance_m               16178 non-null  float64            
 6   is_gap                   11973 non-null  object             
 7   is_jump                  11973 non-null  object             
 8   anomaly_type             98769 non-null  object             
 9   id                       78056 non-null  float64            
 10  snapshot_id              78056 non-null  float64            
 11  msg_time                 78056 

In [ ]:
#I believe this is already resolved
# #Step 2: Also convert timestamp in anomalies to datetime
# df_anomalies_full["timestamp"] = pd.to_datetime(df_anomalies_full["timestamp"], utc=True)


In [66]:
#Create a backup so we don't have to keep rerunning everything
df_anomalies_full_copy = df_anomalies_full.copy()

# #Rebuild from scratch:

# df_anomalies_full_backup = pd.concat([
    # df_anomalies_jumpgap,
    # df_anomalies_stuck,
    # df_anomalies_speed,
    # df_anomalies_backtrack,
    # df_anomalies_repeated,
    # df_anomalies_disappear,
    # df_anomalies_early,
    # df_anomalies_offroute
# ], ignore_index=True)

In [ ]:
# df_anomalies_full = df_anomalies_full_copy

✅ df_anomalies_full_copy has the correct anomaly types:
repeated_points     46200  
stuck_vehicle       30988  
jump_or_gap          9200  ← LOST  
off_route            6767  
backtracking         1616  
impossible_speed      434  ← LOST  
disappearance          42  ← LOST  
early_appearance        1  ← LOST  
But after the route name merge, your current df_anomalies_full is missing the last four types.


####Merge check
(Note: this was resolved in .v8)
3. merge_asof Join Between df_anomalies_full and route_lookup
- merge_result = pd.merge_asof(
    df_anomalies_full.sort_values("timestamp"),
    route_lookup.sort_values("timestamp"),
    on="timestamp",
    by="vehicle_id",
    direction="nearest",
    ...
)

🔍 Potential source of corruption:
- If route_lookup (used to get route IDs for vehicles) was built without tight trip_id / shape_id constraints, it can erroneously associate the wrong route to a vehicle.
- merge_asof can assign the nearest route in time — but this may be a different shape or trip.

🛑 This is a prime suspect for inflated off_route counts.

In [ ]:
#Part of the merge investigation; can be removed later
# # Check how many unique routes are assigned per vehicle
# route_counts = df_anomalies_full.groupby("vehicle_id")["route_short_name"].nunique()
# conflict_vehicles = route_counts[route_counts > 1].index

# print(f"🚨 Vehicles with conflicting routes: {len(conflict_vehicles)}")


In [ ]:
# #Diagnostics Step 1: See how many non-null entries are already in merge_subset["route_short_name"]
# print("🔎 Pre-merge route_short_name stats in merge_subset:")
# print(merge_subset["route_short_name"].value_counts(dropna=False).head(10))
# print("Total non-null route_short_name:", merge_subset["route_short_name"].notna().sum())


In [ ]:
# #Diagnostics Step 2: Perform the exact merge but using a temporary suffix to compare side-by-side
# merged_temp = pd.merge(
#     merge_subset,
#     route_lookup_clean[["vehicle_id", "timestamp", "route_short_name"]],
#     on=["vehicle_id", "timestamp"],
#     how="left",
#     suffixes=("_original", "_lookup")
# )


In [ ]:
# #Diagnostics Step 3: Check how often the two columns agree, disagree, or one is missing
# both_present = merged_temp["route_short_name_original"].notna() & merged_temp["route_short_name_lookup"].notna()
# same = merged_temp["route_short_name_original"] == merged_temp["route_short_name_lookup"]
# disagree = both_present & ~same

# print("✅ Both present:", both_present.sum())
# print("✅ Agreeing values:", (same & both_present).sum())
# print("❌ Disagreeing values:", disagree.sum())


In [ ]:
# #Diagnostics Step 4: View some of the disagreements
# print("⚠️ Sample rows where route_short_name disagrees:")
# display(merged_temp.loc[disagree, ["vehicle_id", "timestamp", "route_short_name_original", "route_short_name_lookup"]].head(10))


In [ ]:
# #Diagnostics Step 5: Inspect which source is more complete
# print("Original non-null:", merged_temp["route_short_name_original"].notna().sum())
# print("Lookup non-null:", merged_temp["route_short_name_lookup"].notna().sum())


In [67]:
df_anomalies_full.head()

,vehicle_id,trip_id,timestamp_prev,timestamp_curr,time_diff_sec,distance_m,is_gap,is_jump,anomaly_type,id,...,route_color,route_text_color,data_quality,timestamp,computed_speed_kph,heading_prev,heading_diff,last_seen,first_seen,distance_from_route_m
0,1902,602646,2025-08-04 00:01:39.522324+00:00,2025-08-04 00:02:09.221975+00:00,29.699651,804.604865,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
1,1902,602646,2025-08-04 00:05:08.721672+00:00,2025-08-04 00:05:39.321847+00:00,30.600175,739.278374,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
2,1902,602646,2025-08-04 00:07:08.921636+00:00,2025-08-04 00:07:39.422088+00:00,30.500452,616.148094,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
3,1902,602646,2025-08-04 00:08:40.421770+00:00,2025-08-04 00:09:10.422470+00:00,30.000700,590.497974,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
4,1902,602646,2025-08-04 00:10:39.520785+00:00,2025-08-04 00:11:09.421332+00:00,29.900547,561.596267,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN


In [68]:
#Let's redo the merging *.v8.2
#Actually... not sure why we were doing this at all
#This is from *.v8 addition, basically imposing exact matches on the merge, opposed to an earlier version that would use merge_asof and perform nearest matching
# 🧼 Step 1: Clean merge inputs
valid_rows = df_anomalies_full["timestamp"].notna()
merge_subset = df_anomalies_full.loc[valid_rows].copy()
route_lookup_clean = route_lookup[route_lookup["timestamp"].notna()].copy()

# 🔧 Ensure matching dtypes
merge_subset["vehicle_id"] = merge_subset["vehicle_id"].astype(str)
route_lookup_clean["vehicle_id"] = route_lookup_clean["vehicle_id"].astype(str)

In [69]:
merge_subset.columns

Index(['vehicle_id', 'trip_id', 'timestamp_prev', 'timestamp_curr',
       'time_diff_sec', 'distance_m', 'is_gap', 'is_jump', 'anomaly_type',
       'id', 'snapshot_id', 'msg_time', 'timestamp_collected', 'longitude',
       'latitude', 'heading', 'speed_mph', 'route_short_name', 'next_stop_id',
       'next_stop_name', 'next_stop_sched_time', 'route_id', 'service_id',
       'trip_headsign', 'trip_short_name', 'direction_id', 'block_id',
       'shape_id', 'wheelchair_accessible', 'bikes_allowed', 'agency_id',
       'route_short_name_static', 'route_long_name', 'route_desc',
       'route_type', 'route_url', 'route_color', 'route_text_color',
       'data_quality', 'timestamp', 'computed_speed_kph', 'heading_prev',
       'heading_diff', 'last_seen', 'first_seen', 'distance_from_route_m'],
      dtype='object')

In [70]:
route_lookup_clean.columns

Index(['vehicle_id', 'timestamp', 'route_short_name'], dtype='object')

In [ ]:
# merge_redux = merge_subset.merge(route_lookup_clean[['vehicle_id', 'timestamp']])

In [ ]:
# #This is the *.v8 addition, basically imposing exact matches on the merge, opposed to an earlier version that would use merge_asof and perform nearest matching
# # 🧼 Step 1: Clean merge inputs
# valid_rows = df_anomalies_full["timestamp"].notna()
# merge_subset = df_anomalies_full.loc[valid_rows].copy()
# route_lookup_clean = route_lookup[route_lookup["timestamp"].notna()].copy()

# # 🔧 Ensure matching dtypes
# merge_subset["vehicle_id"] = merge_subset["vehicle_id"].astype(str)
# route_lookup_clean["vehicle_id"] = route_lookup_clean["vehicle_id"].astype(str)

# # ✅ Drop empty original route column
# merge_subset = merge_subset.drop(columns=["route_short_name"], errors="ignore")

# # 🔁 Step 2: Exact merge
# merged = pd.merge(
#     merge_subset,
#     route_lookup_clean[["vehicle_id", "timestamp", "route_short_name"]],
#     on=["vehicle_id", "timestamp"],
#     how="left"
# )

# # 🔁 Step 3: Patch back into df_anomalies_full
# route_name_map = merged.set_index(merge_subset.index)["route_short_name"]
# df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].combine_first(route_name_map)

# # ✅ Step 4: Final check
# print("✅ Remaining missing route names:", df_anomalies_full["route_short_name"].isna().sum())


#####*.v8.2 250731-1630 note: not sure why we were cleaning the data above
and I think this merging may have caused some issues, so skipping it for now

In [71]:
df_anomalies_full[df_anomalies_full["route_short_name"].isna()].sample(5)


,vehicle_id,trip_id,timestamp_prev,timestamp_curr,time_diff_sec,distance_m,is_gap,is_jump,anomaly_type,id,...,route_color,route_text_color,data_quality,timestamp,computed_speed_kph,heading_prev,heading_diff,last_seen,first_seen,distance_from_route_m
29534,973,601156,NaT,2025-08-04 14:15:09.222296+00:00,NaN,1424.724872,NaN,NaN,impossible_speed,NaN,...,NaN,NaN,NaN,NaT,171.53508,NaN,NaN,NaT,NaT,NaN
11404,977,602401,2025-08-04 22:44:33.721444+00:00,2025-08-04 22:45:04.121597+00:00,30.400153,633.624992,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
7594,466,601715,2025-08-04 00:15:39.322282+00:00,2025-08-04 00:16:09.422101+00:00,30.099819,2991.226912,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
11906,983,602067,2025-08-04 20:16:30.821672+00:00,2025-08-04 20:17:00.322461+00:00,29.500789,577.705614,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
11358,977,602430,2025-08-04 18:53:22.322826+00:00,2025-08-04 18:53:51.722302+00:00,29.399476,885.886520,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN


In [72]:
df_anomalies_full.isna().sum()

,0
vehicle_id,2863
trip_id,2888
timestamp_prev,85852
timestamp_curr,85454
time_diff_sec,89659
distance_m,85454
is_gap,89659
is_jump,89659
anomaly_type,2863
id,23576


##Debugging waypoint 2507251634

In [73]:
df_anomalies_full['route_short_name'].value_counts()

,count
route_short_name,
66,9219
777,7699
766,7445
16,6525
141,5725
157,5215
5,4646
8,3959
155,3824


In [ ]:
#Presently there are no unknowns or nulls, so we don't have to worry about this
# # Treat 'Unknown' as null
# df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].replace("Unknown", pd.NA)

# # Only apply fallback if the lookup column is still present
# if "route_short_name_from_lookup" in df_anomalies_full.columns:
#     df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].combine_first(
#         df_anomalies_full["route_short_name_from_lookup"]
#     )

# # Confirm
# print("Remaining missing route names:", df_anomalies_full["route_short_name"].isna().sum())
# print(df_anomalies_full["route_short_name"].value_counts(dropna=False).head())


In [ ]:
#Same as above cell
# # Confirm null replacement and summary
# df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].replace("Unknown", pd.NA)

# # Summary diagnostics
# print("Remaining missing route names:", df_anomalies_full["route_short_name"].isna().sum())
# print(df_anomalies_full["route_short_name"].value_counts(dropna=False).head())


In [74]:
print("Checkpoint anomaly type breakdown:")
print(df_anomalies_full["anomaly_type"].value_counts(dropna=False))

print("Checkpoint anomaly type breakdown (copy):")
print(df_anomalies_full_copy["anomaly_type"].value_counts(dropna=False))

Checkpoint anomaly type breakdown:
anomaly_type
repeated_points     63374
stuck_vehicle       13737
jump_or_gap         11973
impossible_speed     4205
off_route            3768
NaN                  2863
backtracking          945
disappearance         742
early_appearance       25
Name: count, dtype: int64
Checkpoint anomaly type breakdown (copy):
anomaly_type
repeated_points     63374
stuck_vehicle       13737
jump_or_gap         11973
impossible_speed     4205
off_route            3768
NaN                  2863
backtracking          945
disappearance         742
early_appearance       25
Name: count, dtype: int64


In [75]:
print("Unique types in anomaly_type:")
print(set(type(val) for val in df_anomalies_full["anomaly_type"].unique()))


Unique types in anomaly_type:
{<class 'float'>, <class 'str'>}


In [76]:
print("Full anomaly types (raw):", df_anomalies_full["anomaly_type"].unique())
print("Full anomaly types (with types):", [(val, type(val)) for val in df_anomalies_full["anomaly_type"].unique()])


Full anomaly types (raw): ['jump_or_gap' 'stuck_vehicle' 'impossible_speed' nan 'backtracking'
 'repeated_points' 'disappearance' 'early_appearance' 'off_route']
Full anomaly types (with types): [('jump_or_gap', <class 'str'>), ('stuck_vehicle', <class 'str'>), ('impossible_speed', <class 'str'>), (nan, <class 'float'>), ('backtracking', <class 'str'>), ('repeated_points', <class 'str'>), ('disappearance', <class 'str'>), ('early_appearance', <class 'str'>), ('off_route', <class 'str'>)]


In [77]:
print(selected_detectors)

['jumpgap', 'stuck', 'speed', 'backtrack', 'repeated', 'disappear', 'early', 'offroute']


In [78]:
df_anomalies_full["anomaly_type"].unique()

array(['jump_or_gap', 'stuck_vehicle', 'impossible_speed', nan,
       'backtracking', 'repeated_points', 'disappearance',
       'early_appearance', 'off_route'], dtype=object)

In [79]:
# Step 1: Clean anomaly type column
df_temp = df_anomalies_full.copy()
df_temp["route_short_name"] = df_temp["route_short_name"].fillna("(Missing)")
df_temp["anomaly_type_clean"] = df_temp["anomaly_type"].astype(str)

# Step 2: Build pivot table
pivot = df_temp.pivot_table(
    index="route_short_name",
    columns="anomaly_type_clean",
    aggfunc="size",
    fill_value=0
)

# Step 3: Manually reindex using the full list
expected_columns = [
    "repeated_points", "stuck_vehicle", "jump_or_gap", "off_route",
    "backtracking", "impossible_speed", "disappearance", "early_appearance"
]
pivot = pivot.reindex(columns=expected_columns, fill_value=0)

# Step 4: Diagnostics
print("✅ Total in pivot:", pivot.sum().sum())
print("✅ Total in df:", df_anomalies_full["anomaly_type"].notna().sum())
print("✅ Columns in pivot:", pivot.columns.tolist())


✅ Total in pivot: 98769
✅ Total in df: 98769
✅ Columns in pivot: ['repeated_points', 'stuck_vehicle', 'jump_or_gap', 'off_route', 'backtracking', 'impossible_speed', 'disappearance', 'early_appearance']


Key Fix Summary (for documentation or future review):
- Problem cause: pivot_table(...).reindex(columns=sorted(...)) introduced TypeError due to sorting mixed types (possibly hidden NaNs or object-type weirdness in internal index representation).

- Confirmed all anomaly_type values were str, but reindexing still failed — indicating a more subtle index-level mismatch.

Solution: Explicitly cast anomaly_type to string and reindex with a predefined list of expected columns.

In [80]:
pivot

anomaly_type_clean,repeated_points,stuck_vehicle,jump_or_gap,off_route,backtracking,impossible_speed,disappearance,early_appearance
route_short_name,,,,,,,,
(Missing),0,0,11973,0,0,4205,742,25
1,2039,594,0,95,40,0,0,0
10,2354,415,0,8,36,0,0,0
11,2848,650,0,92,42,0,0,0
140,1435,417,0,83,6,0,0,0
141,4294,1071,0,328,32,0,0,0
155,3026,658,0,111,29,0,0,0
157,4320,684,0,132,79,0,0,0
16,4382,1889,0,180,74,0,0,0


In [81]:
#Second pivot table to try looking at things other than route_short_name
# Step 1: Clean anomaly type column
df_temp2 = df_anomalies_full.copy()
df_temp2["trip_id"] = df_temp2["trip_id"].fillna("(Missing)")
df_temp2["anomaly_type_clean"] = df_temp2["anomaly_type"].astype(str)

# Step 2: Build pivot table
pivot2 = df_temp2.pivot_table(
    index="trip_id",
    columns="anomaly_type_clean",
    aggfunc="size",
    fill_value=0
)

# Step 3: Manually reindex using the full list
expected_columns = [
    "repeated_points", "stuck_vehicle", "jump_or_gap", "off_route",
    "backtracking", "impossible_speed", "disappearance", "early_appearance"
]
pivot2 = pivot2.reindex(columns=expected_columns, fill_value=0)

# Step 4: Diagnostics
print("✅ Total in pivot:", pivot2.sum().sum())
print("✅ Total in df:", df_anomalies_full["anomaly_type"].notna().sum())
print("✅ Columns in pivot:", pivot2.columns.tolist())

✅ Total in pivot: 98769
✅ Total in df: 98769
✅ Columns in pivot: ['repeated_points', 'stuck_vehicle', 'jump_or_gap', 'off_route', 'backtracking', 'impossible_speed', 'disappearance', 'early_appearance']


In [82]:
pivot2

anomaly_type_clean,repeated_points,stuck_vehicle,jump_or_gap,off_route,backtracking,impossible_speed,disappearance,early_appearance
trip_id,,,,,,,,
(Missing),0,0,0,0,0,0,0,25
600305,69,22,10,0,1,4,1,0
600306,69,22,12,0,2,7,1,0
600307,66,22,12,0,2,6,1,0
600308,68,22,13,4,1,3,1,0
...,...,...,...,...,...,...,...,...
603230,113,19,22,14,1,2,1,0
603231,91,4,21,22,3,1,1,0
603265,50,0,12,0,0,2,1,0


####Debuggin in *.v8, to do:
Thanks for the clear context and diagnostic output — this is an important signal, and you're absolutely right to question it.

---

### 🔍 What's Going Wrong

The anomaly types:

* `"jump_or_gap"`, `"impossible_speed"`, `"disappearance"`, `"early_appearance"`

are **only appearing under `(Missing)` routes**, while all known route numbers (like `1`, `16`, `66`) show `0` in those columns.

Yet we know these anomaly types **should be assigned to real routes** in many cases.

---

### 🧠 Root Cause

Those anomalies **likely didn't get their `route_short_name` correctly assigned** during the earlier merge. Here's why:

* Many of these anomaly types are generated from **timestamp-level snapshot comparisons**, often between vehicles’ **first or last records**, or **with subtle time gaps**.
* If the timestamp didn’t match exactly with a route in the `route_lookup`, and you removed `merge_asof`, they **weren’t matched at all**.
* So they fell through with missing `route_short_name`, and got lumped into the `"(Missing)"` category in the pivot.

In short:

> You now get only **exact route matches**, which means **no match = Missing = misattributed anomalies**.

---

### ✅ Recommended Fix

To preserve strict merge logic *but still capture route info for jump/disappearance-type anomalies*, you need a **separate merge strategy** for those rows. Here's how:

---

#### 🔄 Updated Strategy

1. **Retain your current exact-match logic** for most anomalies (good for accuracy).
2. **For rows that still have no route**, use a **second fallback join**:

   * Join on `(vehicle_id, rounded timestamp)` with small ± tolerance via *manual logic*, **not `merge_asof`**.
   * Or pre-fill route for vehicles known to follow the same route across time.

Alternatively, if you're okay adding a **hybrid merge strategy**, then:

* First: `merge` (strict)
* Then: `merge_asof` only on rows that are still `NaN` for route
* Flag those rows as `fallback_match = True` for auditability

---

### 🚧 Immediate Diagnostic You Can Try

Let’s confirm our theory first.

Run this:

```python
missing_routes_df = df_anomalies_full[df_anomalies_full["route_short_name"].isna()]
print("🚫 Missing route entries by anomaly type:")
print(missing_routes_df["anomaly_type"].value_counts())
```

You’ll probably see that all the `jump_or_gap`, `impossible_speed`, etc. live in this `missing_routes_df`.

---

### 🛠️ Optional Patch to Recover These

If you’re willing to use a **limited scope fallback merge**, try this:

```python
# Find rows that still have missing route
missing_route_rows = df_anomalies_full["route_short_name"].isna()

# Attempt fallback merge on these using merge_asof (only here)
fallback_merge = pd.merge_asof(
    df_anomalies_full.loc[missing_route_rows].sort_values("timestamp"),
    route_lookup_clean.sort_values("timestamp"),
    on="timestamp",
    by="vehicle_id",
    direction="nearest",
    tolerance=pd.Timedelta("60s")
)

# Patch only where it's still missing
df_anomalies_full.loc[missing_route_rows, "route_short_name"] = fallback_merge["route_short_name"]
```

⚠️ This preserves your integrity for the main dataset, and **only relaxes the constraint for those otherwise unclassified rows**.

---

Would you like help implementing that fallback merge and visual diagnostic?


** Note: this was debugging notes from ≤*.v6 **

Interestingly, the (Missing) route_short_name has the only instances of jump_or_gap, impossible_speed, disappearance, early_appearance


That’s a critical diagnostic insight — and it likely explains why those anomaly types kept disappearing in earlier pivot tables:

🔍 Root Cause:
By default, pivot_table uses only the combination of index and columns values that exist. If all rows for some anomaly types are associated with a missing route_short_name (NaN), and you drop or exclude these during preprocessing, those types vanish from the pivot.

When you did:

df["route_short_name"] = df["route_short_name"].fillna("(Missing)")

You ensured that:

Those rows were not excluded from the index.

Their anomaly types were included and preserved in the pivot.

✅ Implications:
You must preserve (Missing) rows if you want a complete count across all anomaly types.

If you filter by route, or use .dropna() on route_short_name, anomaly types with only missing routes will be silently excluded.

When you view a pivot per route, types like jump_or_gap, impossible_speed, and disappearance may seem to “disappear” — but they’re actually only found in the (Missing) bucket.

---
To ensure full awareness in reporting or visualizing these anomaly types, do one of the following:

Always include a (Missing) category in route-level pivots.

Add a diagnostic count like:


In [83]:
print("Anomalies with missing route names by type:")
print(df_anomalies_full[df_anomalies_full["route_short_name"].isna()]["anomaly_type"].value_counts())


Anomalies with missing route names by type:
anomaly_type
jump_or_gap         11973
impossible_speed     4205
disappearance         742
early_appearance       25
Name: count, dtype: int64


In [84]:
df_anomalies_full["anomaly_type"].value_counts(dropna=False)


,count
anomaly_type,
repeated_points,63374
stuck_vehicle,13737
jump_or_gap,11973
impossible_speed,4205
off_route,3768
NaN,2863
backtracking,945
disappearance,742
early_appearance,25


#Plot in Folium

In [ ]:
#I don't think this is actually useful at this point
# # Step 1: Create a working copy
# df_plot_base = df_anomalies_full.copy()

# # Step 2: Drop rows with missing coordinates or timestamps
# df_plot_base = df_plot_base.dropna(subset=["latitude", "longitude", "timestamp_collected"])

# # Step 3: Ensure coordinate and timestamp types are correct
# df_plot_base["latitude"] = pd.to_numeric(df_plot_base["latitude"], errors="coerce")
# df_plot_base["longitude"] = pd.to_numeric(df_plot_base["longitude"], errors="coerce")
# df_plot_base["timestamp_collected"] = pd.to_datetime(df_plot_base["timestamp_collected"], errors="coerce", utc=True)

# # Step 4: Optional — Filter for anomalies with known route_short_name
# df_plot_base = df_plot_base[df_plot_base["route_short_name"].notna()]

# # Step 5: Optional — Save a filtered subset by anomaly type, route, or vehicle for visualization
# # You can update these filters dynamically for an interactive tool
# selected_anomaly_type = "stuck_vehicle"
# selected_route = "66"
# # Example: match on anomaly type and route
# df_plot_filtered = df_plot_base[
#     (df_plot_base["anomaly_type"] == selected_anomaly_type) &
#     (df_plot_base["route_short_name"] == selected_route)
# ]

# # Preview result
# print(f"🗺️ Plotting subset for anomaly: '{selected_anomaly_type}' on route '{selected_route}'")
# print(df_plot_filtered[["vehicle_id", "timestamp_collected", "latitude", "longitude", "anomaly_type"]].head())


In [85]:
# Use full anomaly dataframe
df_full2 = df_anomalies_full.copy()

# Ensure lat/lon are floats
df_full2["latitude"] = pd.to_numeric(df_full2["latitude"], errors="coerce")
df_full2["longitude"] = pd.to_numeric(df_full2["longitude"], errors="coerce")

# Ensure timestamp_collected is datetime
df_full2["timestamp_collected"] = pd.to_datetime(df_full2["timestamp_collected"], errors="coerce", utc=True)

# Compute diffs for position and time
df_full2["lat_diff"] = df_full2.groupby("vehicle_id")["latitude"].diff()
df_full2["lon_diff"] = df_full2.groupby("vehicle_id")["longitude"].diff()
df_full2["jump_dist"] = (df_full2["lat_diff"]**2 + df_full2["lon_diff"]**2)**0.5
df_full2["time_diff"] = df_full2.groupby("vehicle_id")["timestamp_collected"].diff().dt.total_seconds()

# Label jump and disappearance events using anomaly_params
df_full2["is_jump"] = df_full2["jump_dist"] > anomaly_params["JUMP_DISTANCE_THRESHOLD"]
df_full2["is_disappearance"] = df_full2["time_diff"] > anomaly_params["DISAPPEARANCE_TIME_THRESHOLD"]

# Create enriched jump DataFrame
jumps_df = df_full2[df_full2["is_jump"]].copy()
jumps_df["lat_prev"] = df_full2.groupby("vehicle_id")["latitude"].shift()
jumps_df["lon_prev"] = df_full2.groupby("vehicle_id")["longitude"].shift()
jumps_df["timestamp_prev"] = df_full2.groupby("vehicle_id")["timestamp_collected"].shift()
jumps_df["timestamp_curr"] = jumps_df["timestamp_collected"]

# Filter to vehicles with enough jumps
jump_counts = jumps_df["vehicle_id"].value_counts()
keep_jumpers = jump_counts[jump_counts >= anomaly_params["MIN_JUMP_COUNT_PER_VEHICLE"]].index
jumps_df = jumps_df[jumps_df["vehicle_id"].isin(keep_jumpers)]

# Base frame of jumpers for disappearance/reappearance
df_jumpers_only = df_full2[df_full2["vehicle_id"].isin(keep_jumpers)].copy()

# Identify disappearance and reappearance points
disappear_df = df_jumpers_only[df_jumpers_only["is_disappearance"]].copy()
is_reappear = df_jumpers_only["is_disappearance"].shift(-1).fillna(False)
reappear_df = df_jumpers_only[is_reappear].copy()

# Summary of Key Changes:
# ✅ Replaces JUMP_DISTANCE_THRESHOLD, DISAPPEARANCE_TIME_THRESHOLD, and MIN_JUMP_COUNT_PER_VEHICLE with their anomaly_params equivalents.
# ✅ Maintains existing structure and logic.
# ✅ Compatible with all downstream Folium plotting layers.


/tmp/ipython-input-794306693.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_reappear = df_jumpers_only["is_disappearance"].shift(-1).fillna(False)


In [ ]:
#I believe this can be depricated
# # that traceback confirms the immediate issue: after merging into shapes_df, the column route_short_name is not present, likely due to:

# # A failed merge (i.e. no matching shape_id values across shapes_df and shape_route_map).

# # The column being named something else (e.g. a merge conflict that renamed it, or a typo).

# # An earlier shapes_df overwrite or clean-up step that removed key columns.

# print("🔍 Columns in shapes_df:", shapes_df.columns.tolist())

# # Check if shape_route_map exists and contains route_short_name
# if 'shape_route_map' in globals():
#     print("✅ Columns in shape_route_map:", shape_route_map.columns.tolist())
#     print("✅ Sample shape_route_map rows:")
#     display(shape_route_map.head())
# else:
#     print("❌ shape_route_map does not exist")

# # Check shape_id overlap
# if 'shapes_df' in globals():
#     print("🧪 shape_id in shapes_df:", shapes_df["shape_id"].nunique())
#     print("🧪 shape_id in shape_route_map:", shape_route_map["shape_id"].nunique())
#     print("🧪 Overlapping shape_ids:", len(set(shapes_df["shape_id"]).intersection(shape_route_map["shape_id"])))


In [86]:
#Step 1: Extract shape_id to route_id from trips_df
shape_to_route = trips[["shape_id", "route_id"]].drop_duplicates()
# trips.txt defines which route uses which shape.
# shape_to_route gives a clean 1:1 or 1:many mapping.

#Step 2: Map route_id to route_short_name from routes_df
route_id_to_name = routes[["route_id", "route_short_name"]]
#Brings in human-readable route names like "1", "10", "66".

#Step 3: Merge to associate shape_id with route_short_name
shape_route_map = shape_to_route.merge(route_id_to_name, on="route_id", how="left")
#You now have a table with: shape_id, route_id, and route_short_name

shapes_copy = shapes.copy()

# Step 4: Drop preexisting route_short_name to avoid merge conflicts
if "route_short_name" in shapes_copy.columns:
    shapes_copy.drop(columns=["route_short_name"], inplace=True)

# Step 4 (retry): Merge into shapes_df
shapes_df = shapes_copy.merge(shape_route_map, on="shape_id", how="left")

# Step 5: Confirm structure
assert shapes_df["route_short_name"].notna().all(), "❌ Some shapes are missing route names!"
print("✅ Final columns in shapes_df:", shapes_df.columns.tolist())
display(shapes_df[["shape_id", "route_short_name"]].drop_duplicates().head())





✅ Final columns in shapes_df: ['shape_id', 'shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence', 'shape_dist_traveled', 'route_id', 'route_short_name']


,shape_id,route_short_name
0,17128,1
251,17129,1
441,17130,10
608,17131,10
685,17132,10


In [ ]:
#I believe this can be depricated
# # Next Step: Clean up unnecessary suffix columns
# # Since the final route_short_name column is already correct, and we don’t need the duplicated *_x and *_y columns, you can safely drop them:
# # Clean up leftover duplicate columns
# cols_to_drop = [col for col in shapes_df.columns if col.endswith("_x") or col.endswith("_y")]
# shapes_df.drop(columns=cols_to_drop, inplace=True)

# # Final confirmation
# print("✅ Cleaned columns in shapes_df:", shapes_df.columns.tolist())
# display(shapes_df[["shape_id", "route_short_name"]].drop_duplicates().head())


Summary of Current shapes_df Columns:
- shape_id: Unique shape identifier
- shape_pt_lat, shape_pt_lon: GPS points to trace the route shape
- shape_pt_sequence: Order of points to draw the shape
- shape_dist_traveled: Optional, often used to interpolate positions
- route_id, route_short_name: Now both included — route_short_name is the human-readable one you’ll want for plotting and filtering

In [ ]:
# # --- Create base map ---
# mymap = Map(location=[35.0844, -106.6504], zoom_start=12)

# # --- Define color map for routes ---
# route_names = sorted(df_anomalies_full['route_short_name'].dropna().unique())
# cmap = plt.get_cmap("tab20", len(route_names))
# color_map = {route: mcolors.to_hex(cmap(i)) for i, route in enumerate(route_names)}

# # --- Plot anomaly markers by route and type ---
# for (route, anomaly), subset in df_anomalies_full.groupby(["route_short_name", "anomaly_type"]):
#     group = FeatureGroup(name=f"{route} – {anomaly}", show=False)
#     for _, row in subset.iterrows():
#         if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
#             popup = (
#                 f"Anomaly: {anomaly}<br>"
#                 f"Route: {route}<br>"
#                 f"Vehicle: {row.get('vehicle_id', 'N/A')}<br>"
#                 f"Timestamp: {row.get('timestamp_collected', 'N/A')}"
#             )
#             CircleMarker(
#                 location=[row["latitude"], row["longitude"]],
#                 radius=4,
#                 color=color_map.get(route, "black"),
#                 fill=True,
#                 fill_opacity=0.9,
#                 popup=popup
#             ).add_to(group)
#     group.add_to(mymap)

# # --- Plot jump lines and markers ---
# if 'jumps_df' in globals() and not jumps_df.empty:
#     for route, group_df in jumps_df.groupby("route_short_name"):
#         jump_line_group = FeatureGroup(name=f"{route} – Jump Lines", show=False)
#         jump_point_group = FeatureGroup(name=f"{route} – Jump Start/End", show=False)
#         for _, row in group_df.iterrows():
#             if all(pd.notna([row["lat_prev"], row["lon_prev"], row["latitude"], row["longitude"]])):
#                 start = [row["lat_prev"], row["lon_prev"]]
#                 end = [row["latitude"], row["longitude"]]
#                 vehicle = row.get("vehicle_id", "N/A")
#                 t_prev = row.get("timestamp_prev", "N/A")
#                 t_curr = row.get("timestamp_curr", "N/A")

#                 PolyLine([start, end], color="orange", weight=2,
#                          tooltip=f"Vehicle {vehicle} jump").add_to(jump_line_group)

#                 CircleMarker(location=start, radius=4, color="blue", fill=True,
#                              fill_opacity=0.9, popup=f"START – {vehicle}<br>{t_prev}").add_to(jump_point_group)
#                 CircleMarker(location=end, radius=4, color="purple", fill=True,
#                              fill_opacity=0.9, popup=f"END – {vehicle}<br>{t_curr}").add_to(jump_point_group)
#         jump_line_group.add_to(mymap)
#         jump_point_group.add_to(mymap)

# # --- Plot disappearances ---
# if 'disappear_df' in globals() and not disappear_df.empty:
#     for route, group_df in disappear_df.groupby("route_short_name"):
#         disappear_group = FeatureGroup(name=f"{route} – Disappearances", show=False)
#         for _, row in group_df.iterrows():
#             if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
#                 Marker(
#                     location=[row["latitude"], row["longitude"]],
#                     icon=Icon(color="red", icon="times-circle", prefix="fa"),
#                     tooltip=f"Vehicle {row.get('vehicle_id')} disappeared<br>{row.get('timestamp_collected')}"
#                 ).add_to(disappear_group)
#         disappear_group.add_to(mymap)

# # --- Plot reappearances ---
# if 'reappear_df' in globals() and not reappear_df.empty:
#     for route, group_df in reappear_df.groupby("route_short_name"):
#         reappear_group = FeatureGroup(name=f"{route} – Reappearances", show=False)
#         for _, row in group_df.iterrows():
#             if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
#                 Marker(
#                     location=[row["latitude"], row["longitude"]],
#                     icon=Icon(color="green", icon="check-circle", prefix="fa"),
#                     tooltip=f"Vehicle {row.get('vehicle_id')} reappeared<br>{row.get('timestamp_collected')}"
#                 ).add_to(reappear_group)
#         reappear_group.add_to(mymap)

# # --- Plot route shapes ---
# if 'shapes_df' in globals() and not shapes_df.empty:
#     for route, group in shapes_df.groupby("route_short_name"):
#         route_group = FeatureGroup(name=f"{route} – Route Shape", show=False)
#         for shape_id, shape_data in group.groupby("shape_id"):
#             shape_data = shape_data.sort_values("shape_pt_sequence")
#             latlons = list(zip(shape_data["shape_pt_lat"], shape_data["shape_pt_lon"]))
#             PolyLine(
#                 locations=latlons,
#                 color=color_map.get(route, "gray"),
#                 weight=2,
#                 opacity=0.6,
#                 popup=f"Route {route} | Shape {shape_id}"
#             ).add_to(route_group)
#         route_group.add_to(mymap)

# # --- Add layer controls ---
# LayerControl(collapsed=False).add_to(mymap)

# # --- Display the map ---
# mymap

# #Save as fully-interactable html file
# from IPython.display import FileLink
# mymap.save("anomaly_map.html")
# display(FileLink("anomaly_map.html"))


In [93]:
# # --- Create base map ---
# mymap = Map(location=[35.0844, -106.6504], zoom_start=12)

# # --- Define color map for routes ---
# route_names = sorted(df_anomalies_full['route_short_name'].dropna().unique(), key=lambda x: int(str(x)))
# cmap = plt.get_cmap("tab20", len(route_names))
# color_map = {route: mcolors.to_hex(cmap(i)) for i, route in enumerate(route_names)}

# # --- Format helper ---
# def format_route(route):
#     return f"Route {str(route).zfill(3)}" if pd.notna(route) else "(Missing)"

# # --- Safe helper for sorting mixed route_short_name types ---
# def safe_route_key(val):
#     try:
#         return int(str(val))
#     except:
#         return float('inf')  # Push non-numeric/missing routes to the end

# # --- Sorted anomaly layers ---
# anomaly_keys = sorted(
#     df_anomalies_full.groupby(["route_short_name", "anomaly_type"]).groups.keys(),
#     key=lambda x: (safe_route_key(x[0]), str(x[1]))
# )

# for route, anomaly in anomaly_keys:
#     subset = df_anomalies_full[
#         (df_anomalies_full["route_short_name"] == route) &
#         (df_anomalies_full["anomaly_type"] == anomaly)
#     ]
#     group = FeatureGroup(name=f"{format_route(route)} – {anomaly}", show=False)
#     for _, row in subset.iterrows():
#         if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
#             popup = (
#                 f"Anomaly: {anomaly}<br>"
#                 f"Route: {route}<br>"
#                 f"Vehicle: {row.get('vehicle_id', 'N/A')}<br>"
#                 f"Timestamp: {row.get('timestamp_collected', 'N/A')}"
#             )
#             CircleMarker(
#                 location=[row["latitude"], row["longitude"]],
#                 radius=4,
#                 color=color_map.get(route, "black"),
#                 fill=True,
#                 fill_opacity=0.9,
#                 popup=popup
#             ).add_to(group)
#     group.add_to(mymap)

# # --- Plot jump lines and markers ---
# if 'jumps_df' in globals() and not jumps_df.empty:
#     for route in sorted(jumps_df["route_short_name"].dropna().unique(), key=lambda x: int(str(x))):
#         group_df = jumps_df[jumps_df["route_short_name"] == route]
#         jump_line_group = FeatureGroup(name=f"{format_route(route)} – Jump Lines", show=False)
#         jump_point_group = FeatureGroup(name=f"{format_route(route)} – Jump Start/End", show=False)
#         for _, row in group_df.iterrows():
#             if all(pd.notna([row["lat_prev"], row["lon_prev"], row["latitude"], row["longitude"]])):
#                 start = [row["lat_prev"], row["lon_prev"]]
#                 end = [row["latitude"], row["longitude"]]
#                 vehicle = row.get("vehicle_id", "N/A")
#                 t_prev = row.get("timestamp_prev", "N/A")
#                 t_curr = row.get("timestamp_curr", "N/A")

#                 PolyLine([start, end], color="orange", weight=2,
#                          tooltip=f"Vehicle {vehicle} jump").add_to(jump_line_group)

#                 CircleMarker(location=start, radius=4, color="blue", fill=True,
#                              fill_opacity=0.9, popup=f"START – {vehicle}<br>{t_prev}").add_to(jump_point_group)
#                 CircleMarker(location=end, radius=4, color="purple", fill=True,
#                              fill_opacity=0.9, popup=f"END – {vehicle}<br>{t_curr}").add_to(jump_point_group)
#         jump_line_group.add_to(mymap)
#         jump_point_group.add_to(mymap)

# # --- Plot disappearances ---
# if 'disappear_df' in globals() and not disappear_df.empty:
#     for route in sorted(disappear_df["route_short_name"].dropna().unique(), key=lambda x: int(str(x))):
#         group_df = disappear_df[disappear_df["route_short_name"] == route]
#         disappear_group = FeatureGroup(name=f"{format_route(route)} – Disappearances", show=False)
#         for _, row in group_df.iterrows():
#             if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
#                 Marker(
#                     location=[row["latitude"], row["longitude"]],
#                     icon=Icon(color="red", icon="times-circle", prefix="fa"),
#                     tooltip=f"Vehicle {row.get('vehicle_id')} disappeared<br>{row.get('timestamp_collected')}"
#                 ).add_to(disappear_group)
#         disappear_group.add_to(mymap)

# # --- Plot reappearances ---
# if 'reappear_df' in globals() and not reappear_df.empty:
#     for route in sorted(reappear_df["route_short_name"].dropna().unique(), key=lambda x: int(str(x))):
#         group_df = reappear_df[reappear_df["route_short_name"] == route]
#         reappear_group = FeatureGroup(name=f"{format_route(route)} – Reappearances", show=False)
#         for _, row in group_df.iterrows():
#             if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
#                 Marker(
#                     location=[row["latitude"], row["longitude"]],
#                     icon=Icon(color="green", icon="check-circle", prefix="fa"),
#                     tooltip=f"Vehicle {row.get('vehicle_id')} reappeared<br>{row.get('timestamp_collected')}"
#                 ).add_to(reappear_group)
#         reappear_group.add_to(mymap)

# # --- Plot route shapes ---
# if 'shapes_df' in globals() and not shapes_df.empty:
#     for route in sorted(shapes_df["route_short_name"].dropna().unique(), key=lambda x: int(str(x))):
#         group = shapes_df[shapes_df["route_short_name"] == route]
#         route_group = FeatureGroup(name=f"{format_route(route)} – Route Shape", show=False)
#         for shape_id, shape_data in group.groupby("shape_id"):
#             shape_data = shape_data.sort_values("shape_pt_sequence")
#             latlons = list(zip(shape_data["shape_pt_lat"], shape_data["shape_pt_lon"]))
#             PolyLine(
#                 locations=latlons,
#                 color=color_map.get(route, "gray"),
#                 weight=2,
#                 opacity=0.6,
#                 popup=f"{format_route(route)} | Shape {shape_id}"
#             ).add_to(route_group)
#         route_group.add_to(mymap)

# # --- Add layer controls ---
# LayerControl(collapsed=False).add_to(mymap)

# # --- Display map ---
# mymap

# # Optional: Save map as HTML
# mymap.save("anomaly_map.html")
# print("✅ Map saved to 'anomaly_map.html'")

KeyboardInterrupt: 

In [ ]:
# # --- Create base map ---
# mymap = Map(location=[35.0844, -106.6504], zoom_start=12)

# # --- Define color map ---
# route_names = sorted(df_anomalies_full['route_short_name'].dropna().unique(), key=lambda x: int(str(x)) if str(x).isdigit() else float('inf'))
# cmap = plt.get_cmap("tab20", len(route_names))
# color_map = {route: mcolors.to_hex(cmap(i)) for i, route in enumerate(route_names)}

# # --- Clean route label ---
# def clean_route(route_val):
#     if pd.isna(route_val):
#         return "Route (Missing)"
#     return f"Route {int(route_val):03}" if str(route_val).isdigit() else f"Route {route_val}"

# # --- Sorted anomaly layers ---
# anomaly_keys = sorted(
#     df_anomalies_full.groupby(["route_short_name", "anomaly_type"]).groups.keys(),
#     key=lambda x: (int(str(x[0])) if pd.notna(x[0]) and str(x[0]).isdigit() else float('inf'), str(x[1]))
# )

# # --- Add anomaly markers ---
# for (route, anomaly_type) in anomaly_keys:
#     subset = df_anomalies_full[
#         (df_anomalies_full["route_short_name"] == route) &
#         (df_anomalies_full["anomaly_type"] == anomaly_type)
#     ]
#     route_label = clean_route(route)
#     anomaly_label = str(anomaly_type).replace("_", " ").title() if pd.notna(anomaly_type) else "(Unknown)"
#     group_name = f"[{route_label}] {anomaly_label}"
#     group = FeatureGroup(name=group_name, show=False)
#     for _, row in subset.iterrows():
#         if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
#             popup = (
#                 f"Anomaly: {anomaly_label}<br>"
#                 f"Route: {route_label}<br>"
#                 f"Vehicle: {row.get('vehicle_id', 'N/A')}<br>"
#                 f"Timestamp: {row.get('timestamp_collected', 'N/A')}"
#             )
#             CircleMarker(
#                 location=[row["latitude"], row["longitude"]],
#                 radius=4,
#                 color=color_map.get(route, "black"),
#                 fill=True,
#                 fill_opacity=0.9,
#                 popup=popup
#             ).add_to(group)
#     group.add_to(mymap)

# # --- Plot jump lines and markers ---
# if 'jumps_df' in globals() and not jumps_df.empty:
#     for route in sorted(jumps_df["route_short_name"].dropna().unique(), key=lambda x: int(str(x)) if str(x).isdigit() else float('inf')):
#         group_df = jumps_df[jumps_df["route_short_name"] == route]
#         route_label = clean_route(route)
#         jump_line_group = FeatureGroup(name=f"[{route_label}] Jump Lines", show=False)
#         jump_point_group = FeatureGroup(name=f"[{route_label}] Jump Points", show=False)
#         for _, row in group_df.iterrows():
#             if all(pd.notna([row["lat_prev"], row["lon_prev"], row["latitude"], row["longitude"]])):
#                 start = [row["lat_prev"], row["lon_prev"]]
#                 end = [row["latitude"], row["longitude"]]
#                 vehicle = row.get("vehicle_id", "N/A")
#                 t_prev = row.get("timestamp_prev", "N/A")
#                 t_curr = row.get("timestamp_curr", "N/A")

#                 PolyLine([start, end], color="orange", weight=2,
#                          tooltip=f"Vehicle {vehicle} jump").add_to(jump_line_group)

#                 CircleMarker(location=start, radius=4, color="blue", fill=True,
#                              fill_opacity=0.9, popup=f"START – {vehicle}<br>{t_prev}").add_to(jump_point_group)
#                 CircleMarker(location=end, radius=4, color="purple", fill=True,
#                              fill_opacity=0.9, popup=f"END – {vehicle}<br>{t_curr}").add_to(jump_point_group)
#         jump_line_group.add_to(mymap)
#         jump_point_group.add_to(mymap)

# # --- Plot disappearances ---
# if 'disappear_df' in globals() and not disappear_df.empty:
#     for route in sorted(disappear_df["route_short_name"].dropna().unique(), key=lambda x: int(str(x)) if str(x).isdigit() else float('inf')):
#         group_df = disappear_df[disappear_df["route_short_name"] == route]
#         route_label = clean_route(route)
#         disappear_group = FeatureGroup(name=f"[{route_label}] Disappearances", show=False)
#         for _, row in group_df.iterrows():
#             if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
#                 Marker(
#                     location=[row["latitude"], row["longitude"]],
#                     icon=Icon(color="red", icon="times-circle", prefix="fa"),
#                     tooltip=f"Vehicle {row.get('vehicle_id')} disappeared<br>{row.get('timestamp_collected')}"
#                 ).add_to(disappear_group)
#         disappear_group.add_to(mymap)

# # --- Plot reappearances ---
# if 'reappear_df' in globals() and not reappear_df.empty:
#     for route in sorted(reappear_df["route_short_name"].dropna().unique(), key=lambda x: int(str(x)) if str(x).isdigit() else float('inf')):
#         group_df = reappear_df[reappear_df["route_short_name"] == route]
#         route_label = clean_route(route)
#         reappear_group = FeatureGroup(name=f"[{route_label}] Reappearances", show=False)
#         for _, row in group_df.iterrows():
#             if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
#                 Marker(
#                     location=[row["latitude"], row["longitude"]],
#                     icon=Icon(color="green", icon="check-circle", prefix="fa"),
#                     tooltip=f"Vehicle {row.get('vehicle_id')} reappeared<br>{row.get('timestamp_collected')}"
#                 ).add_to(reappear_group)
#         reappear_group.add_to(mymap)

# # --- Plot route shapes ---
# if 'shapes_df' in globals() and not shapes_df.empty:
#     for route in sorted(shapes_df["route_short_name"].dropna().unique(), key=lambda x: int(str(x)) if str(x).isdigit() else float('inf')):
#         group = shapes_df[shapes_df["route_short_name"] == route]
#         route_label = clean_route(route)
#         route_group = FeatureGroup(name=f"[{route_label}] Route Shape", show=False)
#         for shape_id, shape_data in group.groupby("shape_id"):
#             shape_data = shape_data.sort_values("shape_pt_sequence")
#             latlons = list(zip(shape_data["shape_pt_lat"], shape_data["shape_pt_lon"]))
#             PolyLine(
#                 locations=latlons,
#                 color=color_map.get(route, "gray"),
#                 weight=2,
#                 opacity=0.6,
#                 popup=f"{route_label} | Shape {shape_id}"
#             ).add_to(route_group)
#         route_group.add_to(mymap)

# # --- Add layer controls ---
# LayerControl(collapsed=False).add_to(mymap)

# # --- Display map ---
# mymap

# # --- Optionally Save ---
# mymap.save("anomaly_map.html")
# print("✅ Map saved to 'anomaly_map.html'")


In [ ]:
# # --- Create base map ---
# mymap = Map(location=[35.0844, -106.6504], zoom_start=12)

# # --- Format route name safely ---
# def format_route(route):
#     return f"Route {str(route).zfill(3)}" if pd.notna(route) else "(Missing)"

# def safe_route_key(val):
#     try:
#         return int(str(val))
#     except:
#         return float('inf')  # Send NaNs or bad values to end

# # --- Define color per anomaly type ---
# anomaly_colors = {
#     "jump_or_gap": "orange",
#     "stuck_vehicle": "red",
#     "impossible_speed": "darkred",
#     "backtracking": "blue",
#     "repeated_points": "purple",
#     "disappearance": "black",
#     "early_appearance": "green",
#     "off_route": "cadetblue",
#     "reappearance": "lime"
# }

# # --- All routes to plot ---
# all_routes = sorted(df_anomalies_full["route_short_name"].dropna().unique(), key=safe_route_key)

# # --- Loop through each route and build a full FeatureGroup ---
# for route in all_routes:
#     route_label = format_route(route)
#     group = FeatureGroup(name=route_label, show=False)

#     # 1. Plot all anomalies for this route
#     subset = df_anomalies_full[df_anomalies_full["route_short_name"] == route]
#     for _, row in subset.iterrows():
#         color = anomaly_colors.get(row["anomaly_type"], "gray")
#         popup = (
#             f"Anomaly: {row.get('anomaly_type')}<br>"
#             f"Vehicle: {row.get('vehicle_id', 'N/A')}<br>"
#             f"Trip: {row.get('trip_id', 'N/A')}<br>"
#             f"Time: {row.get('timestamp_collected', 'N/A')}"
#         )
#         if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
#             CircleMarker(
#                 location=[row["latitude"], row["longitude"]],
#                 radius=4,
#                 color=color,
#                 fill=True,
#                 fill_opacity=0.85,
#                 popup=popup
#             ).add_to(group)

#     # 2. Plot jump lines (if any)
#     if 'jumps_df' in globals() and not jumps_df.empty:
#         jumps_for_route = jumps_df[jumps_df["route_short_name"] == route]
#         for _, row in jumps_for_route.iterrows():
#             if all(pd.notna([row["lat_prev"], row["lon_prev"], row["latitude"], row["longitude"]])):
#                 start = [row["lat_prev"], row["lon_prev"]]
#                 end = [row["latitude"], row["longitude"]]
#                 vehicle = row.get("vehicle_id", "N/A")

#                 # Line
#                 PolyLine([start, end], color="orange", weight=2,
#                          tooltip=f"Vehicle {vehicle} jump").add_to(group)

#                 # Start marker
#                 CircleMarker(location=start, radius=4, color="blue", fill=True,
#                              fill_opacity=0.9,
#                              popup=f"START – {vehicle}<br>{row.get('timestamp_prev')}").add_to(group)

#                 # End marker
#                 CircleMarker(location=end, radius=4, color="purple", fill=True,
#                              fill_opacity=0.9,
#                              popup=f"END – {vehicle}<br>{row.get('timestamp_curr')}").add_to(group)

#     # 3. Plot disappearances
#     if 'disappear_df' in globals() and not disappear_df.empty:
#         disappear_subset = disappear_df[disappear_df["route_short_name"] == route]
#         for _, row in disappear_subset.iterrows():
#             if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
#                 Marker(
#                     location=[row["latitude"], row["longitude"]],
#                     icon=Icon(color="red", icon="times-circle", prefix="fa"),
#                     tooltip=f"Vehicle {row.get('vehicle_id')} disappeared<br>{row.get('timestamp_collected')}"
#                 ).add_to(group)

#     # 4. Plot reappearances
#     if 'reappear_df' in globals() and not reappear_df.empty:
#         reappear_subset = reappear_df[reappear_df["route_short_name"] == route]
#         for _, row in reappear_subset.iterrows():
#             if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
#                 Marker(
#                     location=[row["latitude"], row["longitude"]],
#                     icon=Icon(color="green", icon="check-circle", prefix="fa"),
#                     tooltip=f"Vehicle {row.get('vehicle_id')} reappeared<br>{row.get('timestamp_collected')}"
#                 ).add_to(group)

#     # 5. Add route shape
#     if 'shapes_df' in globals() and not shapes_df.empty:
#         shape_subset = shapes_df[shapes_df["route_short_name"] == route]
#         for shape_id, shape_data in shape_subset.groupby("shape_id"):
#             shape_data = shape_data.sort_values("shape_pt_sequence")
#             latlons = list(zip(shape_data["shape_pt_lat"], shape_data["shape_pt_lon"]))
#             PolyLine(
#                 locations=latlons,
#                 color="gray",
#                 weight=2,
#                 opacity=0.6,
#                 popup=f"{route_label} | Shape {shape_id}"
#             ).add_to(group)

#     # Finalize and add to map
#     group.add_to(mymap)

# # --- Add Layer Control ---
# LayerControl(collapsed=False).add_to(mymap)

# # --- Show and save map ---
# mymap.save("anomaly_map_by_route.html")
# print("✅ Map saved as 'anomaly_map_by_route.html'")
# mymap

In [98]:
# --- Create map ---
mymap = Map(location=[35.0844, -106.6504], zoom_start=12)

# --- Format helpers ---
def format_route(route):
    return f"Route {str(route).zfill(3)}" if pd.notna(route) else "(Missing)"

def safe_route_key(val):
    try:
        return int(str(val))
    except:
        return float("inf")

# --- Anomaly marker color map ---
anomaly_colors = {
    "jump_or_gap": "orange",
    "stuck_vehicle": "red",
    "impossible_speed": "darkred",
    "backtracking": "blue",
    "repeated_points": "purple",
    "disappearance": "black",
    "early_appearance": "green",
    "off_route": "cadetblue",
    "reappearance": "lime"
}

# --- Plot each route x anomaly type as its own group ---
if not df_anomalies_full.empty:
    # Filter out any NaN route_short_name from anomalies (will be handled separately if needed)
    df_anomalies_groupable = df_anomalies_full[df_anomalies_full["route_short_name"].notna()]
    grouped_anomalies = df_anomalies_groupable.groupby(["route_short_name", "anomaly_type"])
    sorted_keys = sorted(grouped_anomalies.groups.keys(), key=lambda x: (safe_route_key(x[0]), str(x[1])))

    for route, anomaly in sorted_keys:
        route_label = format_route(route)
        color = anomaly_colors.get(anomaly, "gray")
        group_name = f"[{route_label}] {anomaly.replace('_', ' ').title()}"
        group = FeatureGroup(name=group_name, show=False)

        subset = grouped_anomalies.get_group((route, anomaly))
        for _, row in subset.iterrows():
            if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
                popup = (
                    f"Anomaly: {anomaly}<br>"
                    f"Vehicle: {row.get('vehicle_id', 'N/A')}<br>"
                    f"Trip: {row.get('trip_id', 'N/A')}<br>"
                    f"Time: {row.get('timestamp_collected', 'N/A')}"
                )
                CircleMarker(
                    location=[row["latitude"], row["longitude"]],
                    radius=4,
                    color=color,
                    fill=True,
                    fill_opacity=0.9,
                    popup=popup
                ).add_to(group)
        group.add_to(mymap)

# --- Plot jumps_df as its own route x "Jump Lines" and "Jump Start/End" groups ---
if "jumps_df" in globals() and not jumps_df.empty:
    for route in sorted(jumps_df["route_short_name"].dropna().unique(), key=safe_route_key):
        route_label = format_route(route)
        group_df = jumps_df[jumps_df["route_short_name"] == route]

        line_group = FeatureGroup(name=f"[{route_label}] Jump Lines", show=False)
        point_group = FeatureGroup(name=f"[{route_label}] Jump Start/End", show=False)

        for _, row in group_df.iterrows():
            if all(pd.notna([row["lat_prev"], row["lon_prev"], row["latitude"], row["longitude"]])):
                start = [row["lat_prev"], row["lon_prev"]]
                end = [row["latitude"], row["longitude"]]
                vehicle = row.get("vehicle_id", "N/A")
                t_prev = row.get("timestamp_prev", "N/A")
                t_curr = row.get("timestamp_curr", "N/A")

                PolyLine([start, end], color="orange", weight=2,
                         tooltip=f"Vehicle {vehicle} jump").add_to(line_group)

                CircleMarker(location=start, radius=4, color="blue", fill=True,
                             fill_opacity=0.9, popup=f"START – {vehicle}<br>{t_prev}").add_to(point_group)
                CircleMarker(location=end, radius=4, color="purple", fill=True,
                             fill_opacity=0.9, popup=f"END – {vehicle}<br>{t_curr}").add_to(point_group)

        line_group.add_to(mymap)
        point_group.add_to(mymap)

# --- Plot disappearances and reappearances ---
for label, df_layer, icon_color, label_suffix in [
    ("disappear_df", disappear_df, "red", "Disappearances"),
    ("reappear_df", reappear_df, "green", "Reappearances")
]:
    if label in globals() and df_layer is not None and not df_layer.empty:
        for route in sorted(df_layer["route_short_name"].dropna().unique(), key=safe_route_key):
            route_label = format_route(route)
            subset = df_layer[df_layer["route_short_name"] == route]
            group = FeatureGroup(name=f"[{route_label}] {label_suffix}", show=False)

            for _, row in subset.iterrows():
                if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
                    Marker(
                        location=[row["latitude"], row["longitude"]],
                        icon=Icon(color=icon_color, icon="info-sign"),
                        tooltip=f"Vehicle {row.get('vehicle_id')}<br>{row.get('timestamp_collected')}"
                    ).add_to(group)
            group.add_to(mymap)

# --- Plot route shapes separately per route ---
if "shapes_df" in globals() and not shapes_df.empty:
    for route in sorted(shapes_df["route_short_name"].dropna().unique(), key=safe_route_key):
        route_label = format_route(route)
        subset = shapes_df[shapes_df["route_short_name"] == route]
        group = FeatureGroup(name=f"[{route_label}] Route Shape", show=False)

        for shape_id, shape_data in subset.groupby("shape_id"):
            shape_data = shape_data.sort_values("shape_pt_sequence")
            latlons = list(zip(shape_data["shape_pt_lat"], shape_data["shape_pt_lon"]))
            PolyLine(
                locations=latlons,
                color="gray",
                weight=2,
                opacity=0.6,
                popup=f"{route_label} | Shape {shape_id}"
            ).add_to(group)

        group.add_to(mymap)

# --- Add layer controls ---
LayerControl(collapsed=False).add_to(mymap)

# --- Display and Save ---
mymap.save("anomaly_map_route_layers.html")
print("✅ Map saved to 'anomaly_map_route_layers.html'")
mymap


KeyboardInterrupt: 